In [1]:
# 0. Colab 파일 업로드 (한 번만 필요)
from google.colab import files
print("필요 파일 업로드")
uploaded = files.upload()

필요 파일 업로드


Saving bus_route.xlsx to bus_route.xlsx
Saving metro_inside.json to metro_inside.json
Saving metro_station.json to metro_station.json
Saving metro_line.json to metro_line.json
Saving bus_stops.xlsx to bus_stops.xlsx


In [2]:
!pip install folium pandas networkx osmnx scipy geopy openpyxl cch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.5/112.5 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.5/100.5 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.4/125.4 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.9/250.9 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 338.0/338.0 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 69.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 101.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 122.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.3/491.3 kB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.4/90.4 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [3]:
# 1. 라이브러리 설치 및 import
import json
import os
import pandas as pd
import networkx as nx
import osmnx as ox
import numpy as np
from osmnx.distance import nearest_nodes
from scipy.spatial import cKDTree
from geopy.geocoders import Nominatim
from itertools import combinations, islice
from math import radians, sin, cos, sqrt, asin
from multiprocessing import Pool
import cch
from geopy.geocoders import Nominatim
from concurrent.futures import ProcessPoolExecutor
import heapq
from itertools import product
from collections import defaultdict
from datetime import datetime, time

In [4]:
# ───────────────────────────────────────────────────────────────────────────────
# 2. 실제 도로망 전체 로드 (network_type='drive')
#    - 서울 전역을 대상으로 OSMnx의 drive 네트워크를 가져옵니다.
#    - 로그가 너무 많으면 아래 주석을 해제하고 설정을 바꿀 수 있습니다.
# ox.config(log_console=True, use_cache=True)

road_graph = ox.graph_from_place("Seoul, South Korea", network_type="drive")

# 2.1. ROAD_COORD 캐시에 각 노드의 좌표(위도/경도) 저장
ROAD_COORD = {}
for node, data in road_graph.nodes(data=True):
    ROAD_COORD[node] = (data['y'], data['x'])

print("도로망 로드 완료")
print("  - 도로망 노드 수:", road_graph.number_of_nodes())
print("  - 도로망 엣지 수:", road_graph.number_of_edges())

도로망 로드 완료
  - 도로망 노드 수: 68314
  - 도로망 엣지 수: 190011


나중에 사용자 위치도 매핑 시켜야함. walk 그래프에. 다만 아직 지도 위에 찍지는 않고, 콘솔에 좌표만 출력하므로 “지도에 매핑” 단계는 다음에 Folium Map 객체 생성 및 Marker 추가를 해주셔야 합니다.

In [6]:
# Step1: 사용자 출발지, 도착지 입력 및 지오코딩

# 1.1 geolocator 초기화
geolocator = Nominatim(user_agent="route_planner")

# 1.2 지오코딩 함수 정의
def get_location(name):
    loc = geolocator.geocode(name, timeout=10)
    if not loc:
        loc = geolocator.geocode(f"{name}, Seoul", timeout=10)
    if not loc:
        raise ValueError(f"'{name}' 위치를 찾을 수 없습니다.")
    return loc.latitude, loc.longitude

# 1.3 사용자에게 출발지와 도착지 명칭 입력받기
start_name = input("출발지 (예: 고려대학교): ")
end_name   = input("도착지 (예: 한남힐스테이트): ")

# 1.4 입력값을 위경도로 변환
start_lat, start_lon = get_location(start_name)
end_lat,   end_lon   = get_location(end_name)

# 1.5 결과 확인용 출력
print(f"출발지 '{start_name}' -> 위도: {start_lat:.6f}, 경도: {start_lon:.6f}")
print(f"도착지 '{end_name}' -> 위도: {end_lat:.6f},   경도: {end_lon:.6f}")

출발지 (예: 고려대학교): 종로3가역
도착지 (예: 한남힐스테이트): 강남역
출발지 '종로3가역' -> 위도: 37.570440, 경도: 126.992324
도착지 '강남역' -> 위도: 37.499935,   경도: 127.026968


Haversine 직선 거리(미터): 10065.997667197036


In [7]:
# 예시 좌표 (출발/도착 좌표)
start_lat, start_lon = 37.570440, 126.992324
end_lat,   end_lon   = 37.497541, 127.026883

u = ox.distance.nearest_nodes(road_graph, start_lon, start_lat)
v = ox.distance.nearest_nodes(road_graph, end_lon,   end_lat)
print("출발 노드 ID:", u, "도착 노드 ID:", v)


출발 노드 ID: 12234274109 도착 노드 ID: 10220955579


In [8]:
# 노드가 제대로 선택되었다면 실제 최단 경로 길이를 구해 본다
try:
    length_m = nx.shortest_path_length(road_graph, u, v, weight='length')
    print("도로망 최단경로 길이(미터):", length_m)
except nx.NetworkXNoPath:
    print("두 노드가 같은 연결 성분에 속해 있지 않습니다. (No path)")


도로망 최단경로 길이(미터): 9845.427578064353


In [9]:
# 2. 유틸리티 함수 정의
def haversine(lat1, lon1, lat2, lon2):
    """두 지점 간 대원거리 계산 (미터)"""
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat, dlon = lat2 - lat1, lon2 - lon1
    a = sin(dlat/2)**2 + cos(lat1)*cos(lat2)*sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    return 6371000 * c

def road_distance(lat1, lon1, lat2, lon2):
    """도로망 최단 경로 길이(미터) 또는 haversine fallback"""
    try:
        u = nearest_nodes(road_graph, lon1, lat1)
        v = nearest_nodes(road_graph, lon2, lat2)
        return nx.shortest_path_length(road_graph, u, v, weight='length')
    except (nx.NetworkXNoPath, Exception):
        return haversine(lat1, lon1, lat2, lon2)

In [10]:
dist = road_distance(start_lat, start_lon, end_lat, end_lon)
print("도로망 기준 거리(미터):", dist)

도로망 기준 거리(미터): 9845.427578064353


In [11]:
# 지하철 관련 전역 변수 (Step3에서 채울 예정)
subway_info = {}    # { station_name: {'lat': float, 'lng': float, 'lines': set([...])}, ... }
subway_graph = {}   # { (station_name, line): [ (neighbor_station, neighbor_line, weight), ... ], ... }

# 버스 관련 전역 변수 (Step4에서 채울 예정)
bus_info = {}       # { stop_id: {'name': str, 'lat': float, 'lng': float}, ... }
bus_graph = {}      # { stop_id: [ (neighbor_stop_id, weight), ... ], ... }

# 글로벌 속도 및 패널티 설정
AVG_SPEED_KMH_GLOBAL = 18.0   # 대중교통 평균 속도 약 18 km/h (필요시 조정)
WALK_SPEED = 5.0              # 보행 속도 5 km/h
WALK_PENALTY = 3.0            # 환승 시 부과되는 보행+대기 패널티 3분


경강선, 인천1호선,인천2호선, 서해선,김포골드라인,용인에버라인,의정부경전철,GTXA,경춘선 제거.

In [12]:
# 1~9호선 및 기타 전철·경전철 노선별로 직접 지정
speed_by_line = {
    '1호선': 28.6, '2호선': 36.5, '3호선': 34.9, '4호선': 30.1,
    '5호선': 32.8, '6호선': 30.0, '7호선': 32.3, '8호선': 32.2,
    '9호선': 26.2,
    '경의중앙선': 41.5, '수인분당선': 41.1,
    '신분당선': 54.9, '신림선': 26.5, '우이신설선': 28.7,
}

In [13]:
# 전역으로 이미 로드되어 있어야 할 것들:
#   road_graph  ──> OSMnx로 만든 도로망 그래프 (networkx.Graph or MultiDiGraph)
#   haversine   ──> (lat1, lon1, lat2, lon2) 간 거리(미터) 계산 함수
#   WALK_SPEED, WALK_PENALTY ──> 보행 속도(km/h), 환승 페널티(분) 등

def build_metro_graph(
    station_json: str,
    line_json: str,
    speed_by_line: dict
):
    """
    station_json: 지하철 역 정보 JSON 파일 경로
    line_json:    지하철 노선 연결 정보 JSON 파일 경로
    speed_by_line: { '1호선': 30.0, '2호선': 32.0, ... }처럼 호선별 평균 속도( km/h ) 딕셔너리

    - station_json의 각 원소 s는 최소한 {'station_cd','name','lat','lng'} 키를 가져야 합니다.
    - line_json의 각 원소 info는 {'line':호선명, 'node':[ {'station':[ {...}, {...} ]}, ... ]} 구조이어야 합니다.
    """
    # 1) JSON 데이터 로드
    stations = json.load(open(station_json, 'r', encoding='utf-8'))['DATA']
    lines    = json.load(open(line_json,    'r', encoding='utf-8'))['DATA']

    # JSON 로드 후, speed_by_line에 정의된 호선만 남긴다
    lines = [info for info in lines if info['line'] in speed_by_line]

    # 2) station_cd → {name, lat, lng} 사전 생성
    station_map = {
        s['station_cd']: {
            'name': s.get('name', s.get('station_nm')),
            'lat':  float(s['lat']),
            'lng':  float(s['lng'])
        }
        for s in stations
    }

    # 3) 남은 lines에 등장하는 station_cd만 사용하도록 필터링
    used_ids = {
        seg['station'][0]['station_cd']
        for info in lines
        for seg in info.get('node', [])
    } | {
        seg['station'][1]['station_cd']
        for info in lines
        for seg in info.get('node', [])
    }
    station_ids = list(used_ids)

    # 4) KDTree용 좌표 배열 생성 (사용자 좌표 → nearest 역 인덱스 용)
    coords_metro = np.array([
        [station_map[_id]['lat'], station_map[_id]['lng']]
        for _id in station_ids
    ])
    station_tree = cKDTree(coords_metro)

    # 5) 도로망 그래프 상의 노드 매핑: station_cd → nearest road_graph 노드 ID
    lats = [station_map[_id]['lat'] for _id in station_ids]
    lons = [station_map[_id]['lng'] for _id in station_ids]
    road_nodes = nearest_nodes(road_graph, lons, lats)
    station_to_road = dict(zip(station_ids, road_nodes))

    # 6) Graph 초기화
    Gm = nx.Graph()

    # 7) 호선별 인접역 엣지 추가
    for info in lines:
        line_name = info['line']
        speed_kmh = speed_by_line.get(line_name, 30.0)
        for seg in info.get('node', []):
            a_cd = seg['station'][0]['station_cd']
            b_cd = seg['station'][1]['station_cd']
            na = station_map[a_cd]
            nb = station_map[b_cd]
            u_node = f"{line_name}_{na['name']}"
            v_node = f"{line_name}_{nb['name']}"

            # 노드 추가
            if not Gm.has_node(u_node):
                Gm.add_node(u_node, station=na['name'], line=line_name, lat=na['lat'], lng=na['lng'])
            if not Gm.has_node(v_node):
                Gm.add_node(v_node, station=nb['name'], line=line_name, lat=nb['lat'], lng=nb['lng'])

            # 도로망 기반 거리 계산
            u_rg = station_to_road[a_cd]
            v_rg = station_to_road[b_cd]
            try:
                dist_m = nx.shortest_path_length(road_graph, u_rg, v_rg, weight='length')
            except Exception:
                dist_m = haversine(na['lat'], na['lng'], nb['lat'], nb['lng'])

            # 시간 weight 계산 (분)
            t_min = dist_m / 1000 / speed_kmh * 60
            Gm.add_edge(u_node, v_node, weight=t_min, mode='metro_edge')

    # 8) 환승 엣지 추가 (같은 station_cd에 따른 역 연결)
    st2nodes = defaultdict(list)
    for node, data in Gm.nodes(data=True):
        st2nodes[data['station']].append(node)

    for nodes_list in st2nodes.values():
        if len(nodes_list) < 2:
            continue
        for i in range(len(nodes_list)):
            for j in range(i+1, len(nodes_list)):
                u, v = nodes_list[i], nodes_list[j]
                lat_u, lng_u = Gm.nodes[u]['lat'], Gm.nodes[u]['lng']
                lat_v, lng_v = Gm.nodes[v]['lat'], Gm.nodes[v]['lng']
                walk_m = haversine(lat_u, lng_u, lat_v, lng_v)
                wt = walk_m / 1000 / WALK_SPEED * 60 + WALK_PENALTY
                Gm.add_edge(u, v, weight=wt, mode='transfer')
                Gm.add_edge(v, u, weight=wt, mode='transfer')

    return Gm, station_ids, station_tree


In [14]:
# 올바른 언패킹 예시
metro_graph, station_ids, station_tree = build_metro_graph(
    'metro_station.json',
    'metro_line.json',
    speed_by_line
)

print(f"지하철 그래프 노드 개수: {metro_graph.number_of_nodes()}")
print(f"지하철 그래프 엣지 개수: {metro_graph.number_of_edges()}")


지하철 그래프 노드 개수: 612
지하철 그래프 엣지 개수: 712


In [15]:
# 1-1) 노드 정보 추출 & DataFrame 생성
nodes_data = [
    {
        'node_key': node,
        'station': data['station'],
        'line':    data['line'],
        'lat':     data['lat'],
        'lng':     data['lng']
    }
    for node, data in metro_graph.nodes(data=True)
]
metro_nodes_df = pd.DataFrame(nodes_data)
print(f"총 메트로 노드 수: {len(metro_nodes_df)}")
display(metro_nodes_df.head(10))

# 1-2) 엣지 정보 추출 & DataFrame 생성
edges_data = [
    {
        'from':       u,
        'to':         v,
        'mode':       data.get('mode'),
        'weight_min': data.get('weight')
    }
    for u, v, data in metro_graph.edges(data=True)
]
metro_edges_df = pd.DataFrame(edges_data)
print(f"총 메트로 엣지 수: {len(metro_edges_df)}")
display(metro_edges_df.head(10))

# 1-3) 특정 역 연결 확인 (예: '신당'역)
station_of_interest = '신당'
mask = metro_nodes_df['station'] == station_of_interest
nodes_of_interest = metro_nodes_df.loc[mask, 'node_key'].tolist()

print(f"\n'{station_of_interest}'역에 해당하는 그래프 노드들:", nodes_of_interest)
for nk in nodes_of_interest:
    nbrs = metro_graph[nk]
    print(f"  • {nk}와 연결된 노드:")
    for nbr, attr in nbrs.items():
        print(f"      - {nbr} (mode={attr['mode']}, {attr['weight']:.2f}분)")


총 메트로 노드 수: 612


,node_key,station,line,lat,lng
0,1호선_연천,연천,1호선,38.100730,127.073720
1,1호선_전곡,전곡,1호선,38.024580,127.071800
2,1호선_청산,청산,1호선,37.981720,127.069120
3,1호선_소요산,소요산,1호선,37.948100,127.061034
4,1호선_동두천,동두천,1호선,37.927878,127.054790
5,1호선_보산,보산,1호선,37.913702,127.057277
6,1호선_동두천중앙,동두천중앙,1호선,37.901885,127.056482
7,1호선_지행,지행,1호선,37.892334,127.055716
8,1호선_덕정,덕정,1호선,37.843188,127.061277
9,1호선_덕계,덕계,1호선,37.818486,127.056486


총 메트로 엣지 수: 712


,from,to,mode,weight_min
0,1호선_연천,1호선_전곡,metro_edge,0.0
1,1호선_전곡,1호선_청산,metro_edge,0.0
2,1호선_청산,1호선_소요산,metro_edge,0.0
3,1호선_소요산,1호선_동두천,metro_edge,0.0
4,1호선_동두천,1호선_보산,metro_edge,0.0
5,1호선_보산,1호선_동두천중앙,metro_edge,0.0
6,1호선_동두천중앙,1호선_지행,metro_edge,0.0
7,1호선_지행,1호선_덕정,metro_edge,0.0
8,1호선_덕정,1호선_덕계,metro_edge,0.0
9,1호선_덕계,1호선_양주,metro_edge,0.0



'신당'역에 해당하는 그래프 노드들: ['2호선_신당', '6호선_신당']
  • 2호선_신당와 연결된 노드:
      - 2호선_상왕십리 (mode=metro_edge, 1.47분)
      - 2호선_동대문역사문화공원 (mode=metro_edge, 1.87분)
      - 6호선_신당 (mode=transfer, 6.73분)
  • 6호선_신당와 연결된 노드:
      - 6호선_청구 (mode=metro_edge, 1.34분)
      - 6호선_동묘앞 (mode=metro_edge, 1.28분)
      - 2호선_신당 (mode=transfer, 6.73분)


In [16]:
def compute_bus_edge(task):
    """
    task: (u_idx, v_idx, lines_set)
    STOP_COORD, STOP_TO_ROAD, road_graph, AVG_SPEED_KMH_GLOBAL 전역 사용.

    # 버스 구간도 OSM 도로망 기반 최단경로로 계산하도록 변경
    """
    u_idx, v_idx, lines = task

    # 1) 허버사인 대신, ‘정류장 idx → 가장 가까운 도로망 노드’ 매핑 사용
    u_rg = STOP_TO_ROAD[u_idx]
    v_rg = STOP_TO_ROAD[v_idx]

    # 2) OSM 도로망에서 최단 경로 길이(미터) 구하기 (weight='length')
    try:
        dist_m = nx.shortest_path_length(road_graph, u_rg, v_rg, weight='length')
    except (nx.NetworkXNoPath, Exception):
        # 도로망 경로가 없으면 허버사인으로 fallback
        y1, x1 = STOP_COORD[u_idx]
        y2, x2 = STOP_COORD[v_idx]
        dist_m = haversine(y1, x1, y2, x2)

    # 3) 거리 → 버스 평균 속도(AVG_SPEED_KMH_GLOBAL)로 분 단위 환산
    t_min = dist_m / 1000 / AVG_SPEED_KMH_GLOBAL * 60
    return u_idx, v_idx, t_min, lines


In [17]:
 def build_bus_graph_optimized(
    stops_excel: str,
    routes_excel: str,
    avg_speed_kmh: float = 18.0
):
    """
    1) stops_excel: 'bus_stops_seoul.xlsx' 경로 (컬럼: ['정류소명','X좌표','Y좌표'])
    2) routes_excel: 'bus_route.xlsx' 경로 (컬럼: ['노선명','정류소명','순번','X좌표','Y좌표'])
    3) avg_speed_kmh: 버스 평균 속도(km/h), 기본 18.0

    - 버스 정류장 간 좌표 매핑 → 잘못 매핑된(거리 차이 큰) 로우들은 자동으로 드롭
    - STOP_TO_ROAD: 각 정류장 인덱스를 OSM 도로망 노드 ID로 매핑하고,
      동일 도로망 노드에 매핑된 정류장이 몇 개 있는지(overlap)를 출력
    - 노드명 = f"{노선명}_{정류소명}"
    - 버스 이동 엣지 생성 (OSM 도로망 최단경로 이용)
    - 보행 환승 엣지 생성 (1,000m 이내, 20분 이내 걸어갈 수 있으면)
    - 반환값: (bus_graph, edge_lines)
      • bus_graph: networkx.Graph (nodes: 노선명_정류소명, edges: mode, weight)
      • edge_lines: { (u_node, v_node): set(노선명) }
    """

    global STOP_COORD, STOP_TO_ROAD, AVG_SPEED_KMH_GLOBAL
    AVG_SPEED_KMH_GLOBAL = avg_speed_kmh

    # ─────────────────────────────────────────────────────────────────────────
    # 1) 정류장 엑셀 불러오기 & KDTree 생성
    stops_df = pd.read_excel(stops_excel, dtype={'X좌표': float, 'Y좌표': float})
    required_cols = {'정류소명','X좌표','Y좌표'}
    if not required_cols.issubset(stops_df.columns):
        raise ValueError(f"stops_excel에 반드시 {required_cols} 컬럼이 있어야 합니다.")

    # 인덱스가 정류장 고유 ID 역할 (0, 1, 2, …)
    stops_df = stops_df.reset_index(drop=True)

    # (1-1) KDTree 생성: stops_df의 (위도, 경도) 배열
    coords_bus = stops_df[['Y좌표','X좌표']].to_numpy()  # shape=(N,2)
    stop_tree  = cKDTree(coords_bus)

    # ─────────────────────────────────────────────────────────────────────────
    # 2) 노선 정보 엑셀 불러오기 & 좌표 기반 매핑 → 잘못된 매핑(drop)
    routes_df = pd.read_excel(routes_excel, dtype={'X좌표': float, 'Y좌표': float})
    required_cols2 = {'버스번호','정류소명','순번','X좌표','Y좌표'}
    if not required_cols2.issubset(routes_df.columns):
        raise ValueError(f"routes_excel에 반드시 {required_cols2} 컬럼이 있어야 합니다.")

    # (2-1) routes_df 각 행의 (위도,경도)를 KDTree 로 쿼리 → nearest stops_df 인덱스 찾기
    route_coords = routes_df[['Y좌표','X좌표']].to_numpy()
    dists, nearest_idx = stop_tree.query(route_coords, k=1)
    routes_df['matched_stop_idx'] = nearest_idx.astype(int)
    routes_df['matched_dist_m'] = [
        haversine(lat1, lon1,
                  stops_df.loc[idx,'Y좌표'], stops_df.loc[idx,'X좌표'])
        for (lat1, lon1), idx in zip(route_coords, nearest_idx)
    ]

    # (2-2) 특정 거리 임계치 이상(여기서는 50m) 차이가 나는 경우 “잘못된 좌표”로 보고 드롭
    DROP_THRESHOLD_M = 50.0
    mask_good = routes_df['matched_dist_m'] <= DROP_THRESHOLD_M
        # ↓ 여기에 삽입 ↓
    # KDTree 매핑 거리가 50m 초과한 로우(드롭될 행) 미리 살펴보기
    unassigned = routes_df[~mask_good]
    print("=== KDTree 매핑 거리가 50m 초과하여 드롭될 예시 10개 ===")
    # Jupyter가 아니면 print()만 사용해도 무방합니다
    display(unassigned[['버스번호','정류소명','matched_dist_m']].head(10))
    print(f"총 드롭된 로우 수: {len(unassigned)}")
    # ↑ 여기까지 삽입 ↑
    num_dropped = (~mask_good).sum()
    if num_dropped > 0:
        print(f">>> **{num_dropped}개의 노선-정류장 로우는 KDTree 매핑 거리가 {DROP_THRESHOLD_M}m 초과하여 드롭됩니다.**")
    routes_df = routes_df[mask_good].copy()

    # ─────────────────────────────────────────────────────────────────────────
    # 3) 버스 그래프 초기화 & 노드 생성 준비
    bus_graph = nx.Graph()
    # edge_lines: “(u_node, v_node) → {노선명 set}”
    edge_lines = defaultdict(set)

    # STOP_COORD: 정류장 idx → (lat, lon)
    STOP_COORD = {
        idx: (row['Y좌표'], row['X좌표'])
        for idx, row in stops_df.iterrows()
    }

    # ─────────────────────────────────────────────────────────────────────────
    # 4) STOP_TO_ROAD (정류장 idx → OSM 도로망 노드 ID) 매핑
    #    (lon, lat) 순서로 nearest_nodes 호출
    lats = stops_df['Y좌표'].values
    lons = stops_df['X좌표'].values
    road_nodes = ox.distance.nearest_nodes(road_graph, lons, lats)
    STOP_TO_ROAD = dict(enumerate(road_nodes))

    # 4-1) 동일한 OSM 노드에 매핑된 정류장(들) 찾기 → overlap 체크
    inv = defaultdict(list)
    for stop_idx, rnode in STOP_TO_ROAD.items():
        inv[rnode].append(stop_idx)

    overlap_count = 0
    overlap_groups = []
    for rnode, idx_list in inv.items():
        if len(idx_list) > 1:
            overlap_groups.append(idx_list)
            overlap_count += len(idx_list)
    if overlap_count > 0:
        print(f">>> **OSM 노드 상에서 겹치는(같은 노드에 매핑된) 정류장 인덱스 총 {overlap_count}개**")
        print("   - 예시 겹침 그룹 (stop_idx 리스트):")
        for grp in overlap_groups:
            print(f"     → {grp}")

    # ─────────────────────────────────────────────────────────────────────────
    # 5) “노선명_정류소명” 노드 & 버스 엣지 생성
    #    → 각 노선별로 ‘순번’ 순으로 정차 정류장 인덱스를 정렬한 뒤 인접 쌍(u_idx, v_idx)에 대해 엣지 추가
    for line_name, grp in routes_df.groupby('버스번호'):
        grp_sorted = grp.sort_values('순번')
        prev_idx = None
        for _, row in grp_sorted.iterrows():
            cur_idx = int(row['matched_stop_idx'])
            stop_name = stops_df.loc[cur_idx, '정류소명']

            # 노드명: "노선명_정류소명"
            this_node = f"{line_name}_{stop_name}"

            # 실제 노드가 없으면 생성 (노드 속성: coord, stop_idx, 노선명)
            if not bus_graph.has_node(this_node):
                lat0, lon0 = STOP_COORD[cur_idx]
                bus_graph.add_node(
                    this_node,
                    coord     = (lat0, lon0),
                    stop_idx  = cur_idx,
                    route     = line_name
                )

            if prev_idx is not None and prev_idx != cur_idx:
                prev_stop_name = stops_df.loc[prev_idx, '정류소명']
                prev_node = f"{line_name}_{prev_stop_name}"

                # 5-1) 버스 구간 거리 계산 (OSM 도로망 경로 길이)
                u_rg = STOP_TO_ROAD[prev_idx]
                v_rg = STOP_TO_ROAD[cur_idx]
                try:
                    dist_m = nx.shortest_path_length(road_graph, u_rg, v_rg, weight='length')
                except (nx.NetworkXNoPath, Exception):
                    # 도로망 연결 없으면 직선거리 fallback
                    lat_u, lon_u = STOP_COORD[prev_idx]
                    lat_v, lon_v = STOP_COORD[cur_idx]
                    dist_m = haversine(lat_u, lon_u, lat_v, lon_v)

                # 5-2) 소요 시간 계산(분) = 거리(m) / 1000 / 속도(km/h) * 60
                t_min = dist_m / 1000 / AVG_SPEED_KMH_GLOBAL * 60

                # 5-3) 양방향 버스 엣지 추가
                bus_graph.add_edge(
                    prev_node, this_node,
                    weight = t_min,
                    mode   = 'bus_edge'
                )
                bus_graph.add_edge(
                    this_node, prev_node,
                    weight = t_min,
                    mode   = 'bus_edge'
                )

                # 5-4) edge_lines에 해당 엣지를 어떤 노선이 쓰는지 기록
                edge_lines[(prev_node, this_node)].add(line_name)
                edge_lines[(this_node, prev_node)].add(line_name)

            prev_idx = cur_idx

    # ─────────────────────────────────────────────────────────────────────────
    # 6) 보행 환승 엣지 추가 (1,000m 반경 / 20분 이내)
    TRANSFER_RADIUS_M = 1000.0
    # 반경(deg) 단순 변환 (위도 1° ≈ 111 km) → 대략적인 검색용
    radius_deg = TRANSFER_RADIUS_M / 111000.0

    # stop_tree는 1번에서 생성한 KDTree
    # coords_bus: (위도,경도) 배열
    for u_idx in range(len(stops_df)):
        # 이 정류장(u_idx) 반경 radius_deg 내 이웃 정류장 인덱스 리스트
        nbrs = stop_tree.query_ball_point(coords_bus[u_idx], r=radius_deg)
        for v_idx in nbrs:
            if v_idx <= u_idx:
                continue

            # 실제 직선 거리(미터)
            lat_u, lon_u = STOP_COORD[u_idx]
            lat_v, lon_v = STOP_COORD[v_idx]
            dist_uv = haversine(lat_u, lon_u, lat_v, lon_v)

            # “걸어서 걸리는 시간(분)”
            walk_time = dist_uv / 1000 / WALK_SPEED * 60

            if walk_time <= 20.0:  # 20분 이내(≒1km)라면 환승 가능
                # u_idx, v_idx에 해당하는 모든 “노선별 노드”를 서로 연결
                # (stop_idx → 어떤 route들이 쓰고 있는지 알아야 함)
                # 따라서 먼저 “이 정류장 u_idx를 쓰는 노드들”을 찾자
                u_nodes = [node for node, data in bus_graph.nodes(data=True)
                           if data.get('stop_idx') == u_idx]
                v_nodes = [node for node, data in bus_graph.nodes(data=True)
                           if data.get('stop_idx') == v_idx]

                for un in u_nodes:
                    for vn in v_nodes:
                        # 이미 버스 엣지가 있으면 skip (버스 엣지가 있으면 mode='bus_edge'이므로)
                        if bus_graph.has_edge(un, vn):
                            continue
                        # u→v 환승 엣지
                        bus_graph.add_edge(
                            un, vn,
                            weight = walk_time + WALK_PENALTY,
                            mode   = 'transfer'
                        )
                        # v→u 환승 엣지
                        bus_graph.add_edge(
                            vn, un,
                            weight = walk_time + WALK_PENALTY,
                            mode   = 'transfer'
                        )

    return bus_graph, edge_lines


In [18]:
# 4.3. build_bus_graph_optimized 함수 실행 및 결과 확인
bus_graph, edge_lines = build_bus_graph_optimized('bus_stops.xlsx', 'bus_route.xlsx', avg_speed_kmh=18.0)

print(f"최종 버스 그래프 노드(정류장) 개수: {bus_graph.number_of_nodes()}")
print(f"최종 버스 그래프 엣지(유니크) 개수: {bus_graph.number_of_edges()}")

=== KDTree 매핑 거리가 50m 초과하여 드롭될 예시 10개 ===


,버스번호,정류소명,matched_dist_m
27,용산01,순천향병원,77.398036
105,5713,비산체육공원,4073.631936
106,5626,비산체육공원,4073.631936
107,5625,비산체육공원,4073.631936
109,702A,선진운수원당차고지.원신4통입구,5703.687423
110,702B,선진운수원당차고지.원신4통입구,5703.687423
114,서초18,선바위역1번출구,1245.482371
115,1152,청구아파트,6430.276762
146,703,자이안트부대,25225.453933
148,7727,설문동,14223.245066


총 드롭된 로우 수: 3230
>>> **3230개의 노선-정류장 로우는 KDTree 매핑 거리가 50.0m 초과하여 드롭됩니다.**
>>> **OSM 노드 상에서 겹치는(같은 노드에 매핑된) 정류장 인덱스 총 6454개**
   - 예시 겹침 그룹 (stop_idx 리스트):
     → [1, 117, 232, 366, 369, 371, 376]
     → [8, 45]
     → [12, 262]
     → [20, 268]
     → [26, 28]
     → [27, 37]
     → [29, 34]
     → [30, 33]
     → [31, 32]
     → [35, 199, 271, 381]
     → [38, 51]
     → [39, 50, 289, 290]
     → [40, 264]
     → [43, 143, 380, 382]
     → [48, 281]
     → [54, 99, 378]
     → [55, 68, 363]
     → [59, 64]
     → [65, 148]
     → [69, 75]
     → [70, 132]
     → [71, 72]
     → [73, 362]
     → [78, 83]
     → [85, 167]
     → [87, 211]
     → [88, 5110]
     → [89, 543]
     → [90, 5092]
     → [91, 98, 293]
     → [100, 101, 142, 276, 372]
     → [102, 373]
     → [111, 116]
     → [112, 144, 302]
     → [113, 114, 375]
     → [115, 227, 365, 370]
     → [120, 249]
     → [121, 243, 364]
     → [145, 296]
     → [147, 253]
     → [151, 2030]
     → [155, 156, 291, 292]
     → [160,

In [19]:
# 예시: "bus_2840" 와 "bus_9421" 의 정류장 인덱스(idx) 가 각각 무엇인지 알아낸 뒤 그 값을 넣어서 테스트

# 1) 정류장 idx → road node ID 출력
print("=== STOP_TO_ROAD 샘플 매핑 확인 ===")
sample_indices = [  # 여기에는 실제 버스 노드 인덱스를 넣어주세요. 예제에서는 2840, 9421 가 아니라
                   # stops_df 에서 할당된 idx(0~n-1) 값이어야 합니다.
                   # 코드 전체에서 가장 먼저 'bus_2840' 처럼 쓰인 건 "bus_graph 노드 키"이지,
                   # STOP_TO_ROAD에서는 정수 idx 그대로를 사용했으니,
                   # stops_df 내부에서 '정류장ID'가 몇 번째 인덱스인지 찾아서 써야 합니다.
                   2840, 9421
                 ]
for idx in sample_indices:
    if idx in STOP_TO_ROAD:
        print(f"정류장 idx {idx} → road_node ID: {STOP_TO_ROAD[idx]}")
    else:
        print(f"정류장 idx {idx} 은 STOP_TO_ROAD에 없습니다.")

# 2) 만약 동일 ID가 찍히면, 두 정류장은 사실상 road_graph 상 같은 노드를 가리키고 있는 것
if len(sample_indices) == 2:
    u_rg = STOP_TO_ROAD.get(sample_indices[0])
    v_rg = STOP_TO_ROAD.get(sample_indices[1])
    print("두 정류장 road-node 동일 여부:", u_rg == v_rg)


=== STOP_TO_ROAD 샘플 매핑 확인 ===
정류장 idx 2840 → road_node ID: 7579130762
정류장 idx 9421 → road_node ID: 10220955579
두 정류장 road-node 동일 여부: False


In [20]:
import pandas as pd

# 2-1) 노드 정보 추출 & DataFrame 생성
nodes_data = [
    {
        'node_key': node,
        'stop_idx': data['stop_idx'],
        'route':    data['route'],
        'lat':      data['coord'][0],
        'lng':      data['coord'][1]
    }
    for node, data in bus_graph.nodes(data=True)
]
bus_nodes_df = pd.DataFrame(nodes_data)
print(f"총 버스 노드 수: {len(bus_nodes_df)}")
display(bus_nodes_df.head(10))

# 2-2) 엣지 정보 추출 & DataFrame 생성
edges_data = [
    {
        'from':       u,
        'to':         v,
        'mode':       data.get('mode'),
        'weight_min': data.get('weight')
    }
    for u, v, data in bus_graph.edges(data=True)
]
bus_edges_df = pd.DataFrame(edges_data)
print(f"총 버스 엣지 수: {len(bus_edges_df)}")
display(bus_edges_df.head(10))

# 2-3) 특정 정류장 연결 확인 (예: 6번째 행의 stop_idx)
stop_idx_of_interest = bus_nodes_df.loc[5, 'stop_idx']

# ① stop_idx로 해당하는 모든 node_key를 뽑아낸 뒤
nodes_of_interest = bus_nodes_df.loc[
    bus_nodes_df['stop_idx'] == stop_idx_of_interest, 'node_key'
].tolist()

print(f"정류장 idx={stop_idx_of_interest}에 해당하는 노드들:", nodes_of_interest)

# ② 실제 존재하는 node_key로 bus_graph 조회
for node_key in nodes_of_interest:
    print(f"\n— {node_key}의 연결 목록 —")
    for nbr, attr in bus_graph[node_key].items():
        print(f"  • {nbr} (mode={attr['mode']}, {attr['weight']:.2f}분)")



총 버스 노드 수: 26647


,node_key,stop_idx,route,lat,lng
0,0017_신용산지하차도,762,0017,37.532590,126.963882
1,0017_용산역,764,0017,37.528867,126.965822
2,0017_용산푸르지오써밋,765,0017,37.527576,126.964760
3,0017_한강대교북단,633,0017,37.524969,126.963571
4,0017_서부이촌동입구,802,0017,37.523864,126.959495
5,0017_이촌2동대림아파트.새남터성지,800,0017,37.526175,126.956323
6,0017_이촌2동주민센터,798,0017,37.527736,126.955064
7,0017_성촌공원.포르쉐센터용산,796,0017,37.530171,126.954607
8,0017_이촌119안전센터.성촌공원,795,0017,37.530080,126.952947
9,0017_원효2동산호아파트후문,804,0017,37.532260,126.950013


총 버스 엣지 수: 2194576


,from,to,mode,weight_min
0,0017_신용산지하차도,0017_용산역,bus_edge,2.012015
1,0017_신용산지하차도,0017_용산전자상가입구,bus_edge,0.713411
2,0017_신용산지하차도,0017_신용산역3번출구,bus_edge,2.995241
3,0017_신용산지하차도,0017_한강대교북단.LG유플러스,transfer,14.423753
4,0017_신용산지하차도,040_한강대교북단.LG유플러스,transfer,14.423753
5,0017_신용산지하차도,500_한강대교북단.LG유플러스,transfer,14.423753
6,0017_신용산지하차도,501_한강대교북단.LG유플러스,transfer,14.423753
7,0017_신용산지하차도,504_한강대교북단.LG유플러스,transfer,14.423753
8,0017_신용산지하차도,506_한강대교북단.LG유플러스,transfer,14.423753
9,0017_신용산지하차도,507_한강대교북단.LG유플러스,transfer,14.423753


정류장 idx=800에 해당하는 노드들: ['0017_이촌2동대림아파트.새남터성지', '2016_이촌2동대림아파트.새남터성지']

— 0017_이촌2동대림아파트.새남터성지의 연결 목록 —
  • 0017_서부이촌동입구 (mode=bus_edge, 1.03분)
  • 0017_이촌2동주민센터 (mode=bus_edge, 0.75분)
  • 0017_이촌2동강변아파트 (mode=bus_edge, 1.36분)
  • 0017_한강대교북단.LG유플러스 (mode=transfer, 10.73분)
  • 040_한강대교북단.LG유플러스 (mode=transfer, 10.73분)
  • 500_한강대교북단.LG유플러스 (mode=transfer, 10.73분)
  • 501_한강대교북단.LG유플러스 (mode=transfer, 10.73분)
  • 504_한강대교북단.LG유플러스 (mode=transfer, 10.73분)
  • 506_한강대교북단.LG유플러스 (mode=transfer, 10.73분)
  • 507_한강대교북단.LG유플러스 (mode=transfer, 10.73분)
  • 605_한강대교북단.LG유플러스 (mode=transfer, 10.73분)
  • 100_한강대교북단.LG유플러스 (mode=transfer, 10.68분)
  • 150_한강대교북단.LG유플러스 (mode=transfer, 10.68분)
  • 151_한강대교북단.LG유플러스 (mode=transfer, 10.68분)
  • 152_한강대교북단.LG유플러스 (mode=transfer, 10.68분)
  • 742_한강대교북단.LG유플러스 (mode=transfer, 10.68분)
  • 750A_한강대교북단.LG유플러스 (mode=transfer, 10.68분)
  • 750B_한강대교북단.LG유플러스 (mode=transfer, 10.68분)
  • 752_한강대교북단.LG유플러스 (mode=transfer, 10.68분)
  • N15_한강대교북단.LG유플러스 (mode=trans

In [21]:
# --- 5.0. 사전 준비: 필요한 전역 변수 및 함수가 이미 정의되었다고 가정 ---
# - metro_graph: Step3에서 생성된 지하철 그래프
# - bus_graph:   Step4에서 생성된 버스 그래프
# - STOP_COORD:  { idx: (lat, lon) } (버스 정류장 인덱스 → 위/경도)
# - WALK_SPEED, WALK_PENALTY: 보행 속도(km/h), 환승 패널티(분)
# - haversine(lat1, lon1, lat2, lon2): 두 지점 간 거리(미터) 계산 함수

# 5.1. 통합 그래프 초기화
integrated_graph = nx.Graph()


In [22]:
# ─── 1. 지하철 노드·엣지 복사 ─────────────────────────────────────────
for node, data in metro_graph.nodes(data=True):
    integrated_graph.add_node(
        node,
        mode='metro',
        station=data.get('station'),
        line=data.get('line'),
        lat=data.get('lat'),
        lng=data.get('lng')
    )

for u, v, data in metro_graph.edges(data=True):
    w = data.get('weight')
    if data.get('mode') == 'transfer':
        integrated_graph.add_edge(u, v, weight=w, mode='transfer')
    else:
        integrated_graph.add_edge(u, v, weight=w, mode='metro_edge')


In [23]:
 # ─── 2. 버스 노드·엣지 복사 ─────────────────────────────────────────
 # bus_graph 노드(key) 그대로 사용, 반드시 'route'까지 복사해야 합니다
for node, data in bus_graph.nodes(data=True):
    integrated_graph.add_node(
        node,
        mode     = 'bus',
        stop_idx = data['stop_idx'],
        route    = data['route'],
        lat      = data['coord'][0],
        lng      = data['coord'][1]
    )
for (u, v), lines in edge_lines.items():
    integrated_graph.add_edge(
        u, v,
        weight = bus_graph[u][v]['weight'],
        mode   = bus_graph[u][v].get('mode','bus_edge'),
        lines  = lines
    )

In [24]:
# ─── 3. 버스↔지하철 환승 엣지 추가 (1 000m 반경) ────────────────────────

# 3-1) 버스 정류장 좌표 배열 + KDTree 준비
# STOP_COORD: { stop_idx: (lat, lon) } 형태
bus_indices   = list(STOP_COORD.keys())
bus_coords    = np.array([STOP_COORD[s] for s in bus_indices])  # shape=(N_bus, 2) → [(lat,lon), ...]
bus_tree      = cKDTree(bus_coords)

# (위도 1° ≈ 111 km) 정도이므로, 1 000 m 반경을 위/경도 단위로 _대략_ 변환
TRANSFER_RADIUS_M = 1000.0
radius_deg = TRANSFER_RADIUS_M / 111000.0  # ≈ 0.009009°

# 3-2) 각 메트로 노드마다 주변 버스 정류장 후보 찾고, 실제 거리 재확인
for m_node, m_data in metro_graph.nodes(data=True):
    metro_lat, metro_lon = m_data['lat'], m_data['lng']
    # KDTree 에서는 (lat, lon) 순서로 쿼리
    # 반경(radius_deg) 내 후보 버스 정류장 인덱스 배열(idx_in_bus_coords_list)을 돌려준다
    idxs_in_bus_array = bus_tree.query_ball_point([metro_lat, metro_lon], r=radius_deg)

    # 실제 환승 가능한 버스 정류장만 골라서 엣지 추가
    for arr_i in idxs_in_bus_array:
        stop_idx = bus_indices[arr_i]           # STOP_COORD의 실제 stop_idx
        bus_lat, bus_lon = STOP_COORD[stop_idx]

        # 실제 거리를 haversine으로 계산
        dist_m = haversine(bus_lat, bus_lon, metro_lat, metro_lon)
        if dist_m <= TRANSFER_RADIUS_M:
            walk_time     = (dist_m / 1000.0) / WALK_SPEED * 60.0
            transfer_time = walk_time + WALK_PENALTY

            bus_node_key  = f"bus_{stop_idx}"
            integrated_graph.add_edge(
                bus_node_key, m_node,
                weight = transfer_time,
                mode   = 'transfer'
            )


In [25]:
# ─── 4. 결과 확인 ─────────────────────────────────────────────────
node_count     = integrated_graph.number_of_nodes()
edge_count     = integrated_graph.number_of_edges()
transfer_edges = [(u, v) for u, v, d in integrated_graph.edges(data=True) if d['mode'] == 'transfer']

print("통합 그래프 노드 개수:", node_count)
print("통합 그래프 엣지 개수:", edge_count)
print("버스↔지하철 환승 엣지 개수:", len(transfer_edges))


통합 그래프 노드 개수: 36714
통합 그래프 엣지 개수: 55038
버스↔지하철 환승 엣지 개수: 24578


In [26]:
# 통합 그래프에서 버스/지하철 환승 엣지만 샘플 확인
count = 0
for u, v, data in integrated_graph.edges(data=True):
    if data.get('mode') == 'transfer':
        print(f"환승 엣지: {u} ↔ {v}, time={data['weight']:.2f}분")
        count += 1
        if count >= 10:
            break

환승 엣지: 1호선_도봉산 ↔ 7호선_도봉산, time=3.62분
환승 엣지: 1호선_도봉산 ↔ bus_3640, time=9.45분
환승 엣지: 1호선_도봉산 ↔ bus_3502, time=6.55분
환승 엣지: 1호선_도봉산 ↔ bus_3501, time=9.51분
환승 엣지: 1호선_도봉산 ↔ bus_3653, time=5.07분
환승 엣지: 1호선_도봉산 ↔ bus_3651, time=4.70분
환승 엣지: 1호선_도봉산 ↔ bus_3652, time=4.87분
환승 엣지: 1호선_도봉산 ↔ bus_3503, time=6.46분
환승 엣지: 1호선_도봉산 ↔ bus_3457, time=12.91분
환승 엣지: 1호선_도봉산 ↔ bus_3713, time=14.41분


In [27]:
import pandas as pd

# 3-1) 노드 정보 추출 & DataFrame 생성
nodes_data = []
for node, data in integrated_graph.nodes(data=True):
    mode = data.get('mode')      # 안전하게 가져오기
    rd = {
        'node_key': node,
        'mode':     mode,
        'lat':      data.get('lat'),
        'lng':      data.get('lng')
    }
    if mode == 'metro':
        rd['station'] = data.get('station')
        rd['line']    = data.get('line')
    elif mode == 'bus':
        rd['stop_idx']= data.get('stop_idx')
        rd['route']   = data.get('route')
    else:
        # mode가 없는(또는 다른) 노드라면 필요에 따라 처리
        rd['note'] = 'unknown mode'
    nodes_data.append(rd)

int_nodes_df = pd.DataFrame(nodes_data)
print(f"총 통합 노드 수: {len(int_nodes_df)}")
display(int_nodes_df.head(10))




총 통합 노드 수: 36714


,node_key,mode,lat,lng,station,line,stop_idx,route,note
0,1호선_연천,metro,38.100730,127.073720,연천,1호선,NaN,NaN,NaN
1,1호선_전곡,metro,38.024580,127.071800,전곡,1호선,NaN,NaN,NaN
2,1호선_청산,metro,37.981720,127.069120,청산,1호선,NaN,NaN,NaN
3,1호선_소요산,metro,37.948100,127.061034,소요산,1호선,NaN,NaN,NaN
4,1호선_동두천,metro,37.927878,127.054790,동두천,1호선,NaN,NaN,NaN
5,1호선_보산,metro,37.913702,127.057277,보산,1호선,NaN,NaN,NaN
6,1호선_동두천중앙,metro,37.901885,127.056482,동두천중앙,1호선,NaN,NaN,NaN
7,1호선_지행,metro,37.892334,127.055716,지행,1호선,NaN,NaN,NaN
8,1호선_덕정,metro,37.843188,127.061277,덕정,1호선,NaN,NaN,NaN
9,1호선_덕계,metro,37.818486,127.056486,덕계,1호선,NaN,NaN,NaN


############ 후보지 생성.

In [28]:
# ——————————————————————————————
#  1) KD-Tree 생성: 위경도 기반 가까운 노드 탐색
# ——————————————————————————————

# (1-1) 위/경도 좌표가 있는 노드만 추출
valid_nodes = []
coords = []
for node, data in integrated_graph.nodes(data=True):
    lat, lng = data.get('lat'), data.get('lng')
    if lat is None or lng is None:
        continue
    valid_nodes.append(node)
    coords.append((lat, lng))

# (1-2) numpy 배열 & KDTree 생성
all_nodes  = valid_nodes
all_coords = np.array(coords)  # shape=(N, 2)
all_tree   = cKDTree(all_coords)


In [29]:
def nearby_multimodal_nodes(user_lat, user_lon, radius_m=1000):
    radius_deg = radius_m / 111000.0
    idxs = all_tree.query_ball_point([user_lat, user_lon], r=radius_deg)
    print(f"→ radius_deg={radius_deg:.5f}, found {len(idxs)} candidate indices")
    for i in idxs[:10]:
        print("   sample node:", all_nodes[i], "coord:", all_coords[i])
    results = []
    for i in idxs:
        node = all_nodes[i]
        lat2, lon2 = integrated_graph.nodes[node]['lat'], integrated_graph.nodes[node]['lng']
        t_min = walk_time_via_road(user_lat, user_lon, lat2, lon2)
        results.append((node, t_min))
    results.sort(key=lambda x: x[1])
    return results


In [30]:
# ——————————————————————————————
#  2) 도로망 기반 보행시간 계산
# ——————————————————————————————

def walk_time_via_road(lat1, lon1, lat2, lon2):
    """
    두 지점 간 도로망(shortest-path) 보행시간 계산 (분 단위)
    - 경로가 없으면 허버사인 거리로 fallback
    """
    try:
        # 도로망에서 가장 가까운 노드 매핑
        u = nearest_nodes(road_graph, lon1, lat1)
        v = nearest_nodes(road_graph, lon2, lat2)
        # 최단 경로 길이 (미터)
        dist_m = nx.shortest_path_length(road_graph, u, v, weight='length')
    except (nx.NetworkXNoPath, Exception):
        # 네트워크 연결 불가 시 허버사인 거리 사용
        dist_m = haversine(lat1, lon1, lat2, lon2)
    # 거리 → 시간(분) 환산
    return (dist_m / 1000.0) / WALK_SPEED * 60.0


In [31]:
# 1) 계절 계산 함수
# ——————————————————————————————
def get_current_season(now=None):
    """
    now.month를 기준으로 계절 반환
      3~5월   봄 ('spring')
      6~8월   여름 ('summer')
      9~11월  가을 ('fall')
      12,1,2월 겨울 ('winter')
    """
    if now is None:
        now = datetime.now()
    m = now.month
    if 3 <= m <= 5:
        return 'spring'
    if 6 <= m <= 8:
        return 'summer'
    if 9 <= m <= 11:
        return 'fall'
    return 'winter'

# 모드별 허용 계절(필요시 수정)
SEASON_SERVICE = {
    'metro': ['spring','summer','fall','winter'],
    'bus':   ['spring','summer','fall','winter']
}


In [32]:
# ──────────────────────────────────────────
# 2) 정교한 운행 시간 로직 함수
# ──────────────────────────────────────────

def is_metro_operating(now=None):
    if now is None:
        now = datetime.now().time()
    # 05:30 ≤ now < 24:00  또는  00:00 ≤ now < 00:30
    return (now >= time(5,30)) or (now < time(0,30))

from zoneinfo import ZoneInfo  # 파이썬 3.9+

def is_bus_operating(node, now=None):
    """
    일반버스: 05:30 <= now <= 23:59:59
    N-야간버스: 00:00 <= now < 05:30
    """
    if now is None:
        # 한국시간 기준으로 now
        now = datetime.now(ZoneInfo("Asia/Seoul")).time()

    # integrated_graph에 저장된 route 속성 가져오기
    raw = integrated_graph.nodes[node]['route']
    # raw가 str이면 [raw], 그 외(리스트 등)면 list(raw)
    lines = [raw] if isinstance(raw, str) else list(raw)

    # 일반버스 리스트, 야간버스 리스트
    general = [r for r in lines if not r.startswith('N')]
    night   = [r for r in lines if     r.startswith('N')]

    # 1) 일반버스 운행 여부
    if general and time(5,30) <= now <= time(23,59,59):
        return True
    # 2) 야간버스 운행 여부
    if night and time(0,0) <= now < time(5,30):
        return True

    return False


In [33]:
# ——————————————————————————————
# (B) bus_nodes: 'mode'=='bus' AND 'route' 속성이 있는 노드만
# ——————————————————————————————
bus_nodes = [
    n for n,d in integrated_graph.nodes(data=True)
    if d.get('mode')=='bus' and d.get('route') is not None
]

# 헬퍼: node에 걸리는 노선 리스트로 정리
def get_bus_lines(node):
    raw = integrated_graph.nodes[node].get('route') or []
    return [raw] if isinstance(raw, str) else list(raw)

# ——————————————————————————————
# (C) 일반버스 전용 노드 하나
# ——————————————————————————————
gen_node = next(
    n for n in bus_nodes
    if get_bus_lines(n)              # 노선 하나라도 있고
    and all(not r.startswith('N') for r in get_bus_lines(n))  # 모두 일반
)

# ——————————————————————————————
# (D) N-야간버스 전용 노드 하나
# ——————————————————————————————
night_node = next(
    n for n in bus_nodes
    if get_bus_lines(n)
    and all(r.startswith('N') for r in get_bus_lines(n))  # 모두 N-야간
)

# ——————————————————————————————
# (E) 테스트 출력
# ——————————————————————————————
test_times = [
    time(0,0),  time(0,30), time(2,0),
    time(4,0),  time(5,0),  time(5,30),
    time(10,0), time(23,0), time(13,30)
]

print("Time  | Metro | GenBus | NightBus")
for t in test_times:
    m = '●' if is_metro_operating(t) else '×'
    g = '●' if is_bus_operating(gen_node, t) else '×'
    x = '●' if is_bus_operating(night_node, t) else '×'
    print(f"{t:%H:%M} |   {m}   |    {g}   |     {x}")

Time  | Metro | GenBus | NightBus
00:00 |   ●   |    ×   |     ●
00:30 |   ×   |    ×   |     ●
02:00 |   ×   |    ×   |     ●
04:00 |   ×   |    ×   |     ●
05:00 |   ×   |    ×   |     ●
05:30 |   ●   |    ●   |     ×
10:00 |   ●   |    ●   |     ×
23:00 |   ●   |    ●   |     ×
13:30 |   ●   |    ●   |     ×


In [34]:
def filter_candidates_by_availability(candidates, now=None):
    """
    - candidates: [(node_key, walk_min), …]
    - 반환: 현재 계절·운행 중인 모드만 필터링
    """
    if now is None:
        now = datetime.now().time()
    season = get_current_season()

    print(f"\n[필터 시작] season={season}, now={now:%H:%M:%S}, 후보 수={len(candidates)}")
    valid = []
    for node, walk in candidates:
        mode = integrated_graph.nodes[node].get('mode')
        # 1) 계절 허용 여부
        if season not in SEASON_SERVICE.get(mode, []):
            print(f"  ✗ {node}({mode}) — 계절 불허 ({season})")
            continue

        # 2) 시간대별 운행 여부
        if mode == 'metro':
            ok = is_metro_operating(now)
            print(f"  {'●' if ok else '✗'} {node}(metro) — is_metro_operating? {ok}")
            if not ok:
                continue

        elif mode == 'bus':
            ok = is_bus_operating(node, now)
            print(f"  {'●' if ok else '✗'} {node}(bus)   — is_bus_operating? {ok}")
            if not ok:
                continue

        else:
            # mode 잘못 들어갔을 경우
            print(f"  ⚠ {node} — 알 수 없는 mode={mode}")
            continue

        # 여기까지 통과한 후보만 valid에 추가
        valid.append((node, walk))

    print(f"[필터 완료] 살아남은 후보 수={len(valid)}\n")
    return valid


In [35]:
# (1) 현재 시각
from datetime import datetime, timedelta

# UTC 기준 +9시간
now = (datetime.utcnow() + timedelta(hours=9)).time()


# (2) KDTree → 도로망 보행시간 계산으로 후보지 생성
start_candidates = nearby_multimodal_nodes(start_lat, start_lon,  radius_m=1000)
end_candidates   = nearby_multimodal_nodes(end_lat,   end_lon,    radius_m=1000)

# (3) 운행·계절 필터 적용
start_candidates = filter_candidates_by_availability(start_candidates, now)
end_candidates   = filter_candidates_by_availability(end_candidates,   now)

# (4) 결과 확인
print("출발 후보 (운행·계절 필터, 상위 10):", start_candidates[:10])
print("도착 후보 (운행·계절 필터, 상위 10):", end_candidates[:10])


→ radius_deg=0.00901, found 280 candidate indices
   sample node: 202_을지로입구.로얄호텔 coord: [ 37.56587574 126.98539028]
   sample node: N73_을지로입구.로얄호텔 coord: [ 37.56587574 126.98539028]
   sample node: 100_을지로입구.로얄호텔 coord: [ 37.56587574 126.98539028]
   sample node: 261_을지로입구.로얄호텔 coord: [ 37.56587574 126.98539028]
   sample node: 105_을지로입구.로얄호텔 coord: [ 37.56587574 126.98539028]
   sample node: 01B_을지로입구.로얄호텔 coord: [ 37.56587574 126.98539028]
   sample node: N30_을지로입구.로얄호텔 coord: [ 37.56587574 126.98539028]
   sample node: 472_을지로입구.로얄호텔 coord: [ 37.56587574 126.98539028]
   sample node: 152_을지로입구.로얄호텔 coord: [ 37.56587574 126.98539028]
   sample node: 140_국가인권위.안중근활동터 coord: [ 37.56435047 126.98785152]
→ radius_deg=0.00901, found 254 candidate indices
   sample node: 서초03_삼성레포츠 coord: [ 37.49220643 127.02081643]
   sample node: 서초03_삼성래미안아파트 coord: [ 37.49496599 127.01849311]
   sample node: 서초10_삼성래미안아파트 coord: [ 37.49496599 127.01849311]
   sample node: 서초21_서초2동주민센터.서이초등학교 coord: [ 

###########

In [36]:
def get_fastest_routes_optimized(G, origin_candidates, dest_candidates,
                                 k=10, max_transfers=4):
    """
    G: 통합 그래프 (metro+bus+transfer)
    origin_candidates: [(origin_node, origin_walk_min), …]
    dest_candidates:   [(dest_node,   dest_walk_min),   …]
    k: 상위 k개 결과 반환
    max_transfers: 허용할 최대 환승 횟수
    """
    fastest_list = []

    for origin_node, origin_walk in origin_candidates:
        # 1) 한 번만 Dijkstra
        distances, paths = nx.single_source_dijkstra(
            G, origin_node, weight='weight'
        )

        for dest_node, dest_walk in dest_candidates:
            if dest_node not in distances:
                continue

            path        = paths[dest_node]
            travel_time = distances[dest_node]
            total_time  = travel_time + origin_walk + dest_walk

            # ——— 시작/끝 “transfer” 엣지 제외 ———
            if len(path) >= 2:
                # 시작
                if G[path[0]][path[1]].get('mode') == 'transfer':
                    continue
                # 끝
                if G[path[-2]][path[-1]].get('mode') == 'transfer':
                    continue

            # 2) 환승 횟수 계산
            transfer_count = 0
            prev_mode, prev_line = None, None
            edges = list(zip(path, path[1:]))

            for idx, (u, v) in enumerate(edges):
                m = G[u][v].get('mode')
                # 중간 환승 엣지
                if m == 'transfer':
                    if 0 < idx < len(edges)-1:
                        transfer_count += 1
                    prev_mode, prev_line = 'transfer', None
                    continue
                # metro_edge: 호선 변경 시
                if m == 'metro_edge':
                    cur_line = G.nodes[u]['line']
                    if prev_mode=='metro_edge' and prev_line!=cur_line:
                        transfer_count += 1
                    prev_mode, prev_line = 'metro_edge', cur_line
                    continue
                # bus_edge: 노선 변경 시
                if m == 'bus_edge':
                    cur_route = next(iter(G[u][v].get('lines', [])), None)
                    if prev_mode=='bus_edge' and prev_line!=cur_route:
                        transfer_count += 1
                    prev_mode, prev_line = 'bus_edge', cur_route
                    continue
                prev_mode, prev_line = m, None

            # 3) 환승 제한
            if transfer_count > max_transfers:
                continue

            fastest_list.append({
                'path':      path,
                'time_min':  total_time,
                'transfers': transfer_count,
                'walk_min':  origin_walk + dest_walk
            })

    # 상위 k개만
    fastest_list.sort(key=lambda x: x['time_min'])
    return fastest_list[:k]


In [37]:
def dedup_bus_only(candidates):
    """
    • metro 노드는 전부 유지
    • bus  노드는 같은 route(버스번호) 그룹에서 walk 시간이 최소인 것만 남김
    """
    kept = []
    seen_bus = {}   # route → (node, walk)
    for node, walk in candidates:
        data = integrated_graph.nodes[node]
        mode = data.get('mode')
        if mode == 'bus':
            route = data.get('route')
            # 처음이거나 더 짧은 보행시간이면 갱신
            if route not in seen_bus or walk < seen_bus[route][1]:
                seen_bus[route] = (node, walk)
        else:
            # metro는 무조건 추가
            kept.append((node, walk))
    # bus 중복 제거된 항목들을 뒤에 붙이기
    kept.extend(seen_bus.values())
    return kept


In [38]:
# ——————————————————————————————
# 5) **여기서** dedup_by_station_and_busroute 적용
# ——————————————————————————————
start_candidates = dedup_bus_only(start_candidates)
end_candidates   = dedup_bus_only(end_candidates)

# (중복 제거된 최종 후보 확인)
print("\n▶ Dedup된 출발 후보:", start_candidates)
print("▶ Dedup된 도착 후보:",   end_candidates)

# 6) 최단 시간 경로 뽑기
routes = get_fastest_routes_optimized(
    integrated_graph,
    start_candidates,
    end_candidates,
    k=10,
    max_transfers=4
)

# 7) 결과 출력
for i, r in enumerate(routes, 1):
    print(f"\n{i}위: 총시간={r['time_min']:.1f}분 "
          f"(보행={r['walk_min']:.1f}분, 환승={r['transfers']}회)")
    print("  →", " → ".join(r['path']))



▶ Dedup된 출발 후보: [('1호선_종로3가', np.float64(0.017749571362737106)), ('3호선_종로3가', np.float64(1.2952134738131797)), ('5호선_종로3가', np.float64(4.194358523874303)), ('5호선_을지로4가', np.float64(10.049091412857074)), ('2호선_을지로4가', np.float64(11.01495205389755)), ('3호선_을지로3가', np.float64(13.302344320797138)), ('2호선_을지로3가', np.float64(14.209931287710392)), ('종로12_종로3가역9번출구', np.float64(0.5231744087494039)), ('273_종로4가.종묘', np.float64(2.831018704562441)), ('7212_종로4가.종묘', np.float64(2.831018704562441)), ('720_종로4가.종묘', np.float64(2.831018704562441)), ('140_종로4가.종묘', np.float64(2.831018704562441)), ('721_종로4가.종묘', np.float64(2.831018704562441)), ('270_종로4가.종묘', np.float64(2.831018704562441)), ('271_종로4가.종묘', np.float64(3.317118151357348)), ('260_종로4가.종묘', np.float64(3.317118151357348)), ('8101_종로4가.종묘', np.float64(3.317118151357348)), ('160_종로4가.종묘', np.float64(3.317118151357348)), ('370_종로4가.종묘', np.float64(3.317118151357348)), ('103_종로4가.종묘', np.float64(3.317118151357348)), ('101_종로4가.종묘', np.float64

#정시성 함수 만들기.
- 사고는 최근 5년, 늦은건 1년 기준
- 클수록 안좋음.

In [39]:
# 0. Colab 파일 업로드 (한 번만 필요)
uploaded = files.upload()

Saving metro_late.xlsx to metro_late.xlsx
Saving bus_accident.xlsx to bus_accident.xlsx
Saving bus_late.xlsx to bus_late.xlsx
Saving metro_accident.json to metro_accident.json


In [40]:
# 1) Metro 정시성 데이터
metro_late_df = pd.read_excel('metro_late.xlsx', dtype=str)
print("===== metro_late.xlsx =====")
print("Columns:", list(metro_late_df.columns))
print(metro_late_df.head(10), "\n")

# 2) Bus 사고 데이터
bus_accident_df = pd.read_excel('bus_accident.xlsx', dtype=str)
print("===== bus_accident.xlsx =====")
print("Columns:", list(bus_accident_df.columns))
print(bus_accident_df.head(10), "\n")

# 3) Bus 지연 데이터
bus_late_df = pd.read_excel('bus_late.xlsx', dtype=str)
print("===== bus_late.xlsx =====")
print("Columns:", list(bus_late_df.columns))
print(bus_late_df.head(10), "\n")

# 4) Metro 사고(또는 고장) JSON
with open('metro_accident.json', 'r', encoding='utf-8') as f:
    metro_accident = json.load(f)

# JSON 구조에 따라 DATA 키가 있을 경우
if isinstance(metro_accident, dict) and 'DATA' in metro_accident:
    metro_accident_df = pd.json_normalize(metro_accident['DATA'])
else:
    metro_accident_df = pd.json_normalize(metro_accident)

print("===== metro_accident.json =====")
print("Columns:", list(metro_accident_df.columns))
print(metro_accident_df.head(10))

===== metro_late.xlsx =====
Columns: ['지연일자', '노선', '지연시간대', '최대지연시간(분)']
         지연일자   노선    지연시간대 최대지연시간(분)
0  2023-09-01  2호선   첫차~09시        10
1  2023-09-01  2호선   첫차~09시         5
2  2023-09-01  1호선   첫차~09시        15
3  2023-09-01  1호선   첫차~09시        15
4  2023-09-01  4호선   첫차~09시        10
5  2023-09-01  4호선  09시~18시        10
6  2023-09-01  3호선   첫차~09시        10
7  2023-09-01  3호선   첫차~09시        10
8  2023-09-04  3호선   첫차~09시        10
9  2023-09-04  4호선   첫차~09시        10 

===== bus_accident.xlsx =====
Columns: ['버스ID', '노선명', '좌표X', '좌표Y', '구간명']
         버스ID       노선명          좌표X         좌표Y                           구간명
0  ﻿106024501      ﻿01A  ﻿126.979078  ﻿37.576109               ﻿경복궁~경복궁_율곡로_진입
1  ﻿108032029      ﻿104  ﻿127.022016  ﻿37.600707                  ﻿이북오도신문사~복선교
2  ﻿108032029      ﻿104  ﻿126.996654  ﻿37.579082             ﻿창경궁.서울대학교병원~원남빌딩
3  ﻿110058126      ﻿105  ﻿127.063532  ﻿37.659573         ﻿상계주공7단지.광림교회앞~7단지영업소
4  ﻿108045531      ﻿109  ﻿127.01290

In [41]:
!pip install statsmodels
from statsmodels.stats.proportion import proportion_confint

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 101.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.9/232.9 kB 15.7 MB/s eta 0:00:00


In [42]:
# metro_late_df를 불러온 이후, '최대지연시간(분)'을 숫자형으로 변환
metro_late_df['최대지연시간(분)'] = pd.to_numeric(
    metro_late_df['최대지연시간(분)'],
    errors='coerce'    # 변환 불가능한 값은 NaN으로 처리
)

# (선택) NaN이 생겼다면, 드롭하거나 0으로 채워 줍니다
metro_late_df = metro_late_df.dropna(subset=['최대지연시간(분)'])

In [43]:
# ---------------------------------
# A) 지하철 지연 통계(metro_stats) & 사고 로그(log_acc_map) 계산
# ---------------------------------
# A1) 지연 데이터 불러오기 및 전처리
metro_late_df['지연일자'] = pd.to_datetime(metro_late_df['지연일자'], errors='coerce')
metro_late_df['최대지연시간(분)'] = pd.to_numeric(metro_late_df['최대지연시간(분)'], errors='coerce')
metro_late_df = metro_late_df.dropna(subset=['지연일자','최대지연시간(분)'])

# A2) 계절 매핑 함수
def get_season(dt):
    m = dt.month
    if 3 <= m <= 5: return 'spring'
    if 6 <= m <= 8: return 'summer'
    if 9 <= m <= 11: return 'autumn'
    return 'winter'
metro_late_df['season'] = metro_late_df['지연일자'].apply(get_season)

# A3) 계절별 총 일수
season_days = {'winter':90,'spring':92,'summer':92,'autumn':91}

# ---------------------------------
# A4) 노선·계절별 delay_days, avg_delay 계산 (수정)
# ---------------------------------
metro_stats = (
    metro_late_df.groupby(['노선','season'])
    .agg(
        delay_days=('지연일자', lambda x: x.dt.date.nunique()),
        avg_delay=('최대지연시간(분)', 'mean')
    )
    .reset_index()
)

# ① 계절별 총 일수 매핑
metro_stats['total_days'] = metro_stats['season'].map(season_days)

# ② on_time_days, reliability 계산
metro_stats['on_time_days'] = metro_stats['total_days'] - metro_stats['delay_days']
metro_stats['reliability'] = metro_stats['on_time_days'] / metro_stats['total_days']

# ③ avg_delay 스케일링: "5개의 구간으로 나눈 값"을 5로 나눠준다
metro_stats['avg_delay'] = metro_stats['avg_delay'] / 5

# 최종 컬럼 순서 정리
metro_stats = metro_stats[['노선','season','avg_delay','reliability']]


# ——————————————————————————————————————————————————
# A5) 지하철 사고 데이터 → 발생확률(acc_prob_map) 계산 (수정)
# ——————————————————————————————————————————————————

# 1) 이미 로드된 metro_accident_df 복사
acc_df = metro_accident_df.copy()

# 2) station_clean: '역' 접미사 제거해 그래프의 station 필드와 매칭
acc_df['station_clean'] = (
    acc_df['역명']
    .str.replace(r'역$','',regex=True)
    .str.strip()
)

# 3) 사고횟수 int 변환
acc_df['사고횟수'] = pd.to_numeric(acc_df['사고횟수'], errors='coerce').fillna(0).astype(int)

# 4) station_clean → 사고횟수 맵 생성
acc_map = acc_df.set_index('station_clean')['사고횟수'].to_dict()

# 5) 통합 그래프에 있지만 acc_map에 없는 역은 0으로 추가
for node, data in integrated_graph.nodes(data=True):
    if data.get('mode')=='metro':
        st = data['station']
        if st not in acc_map:
            acc_map[st] = 0

# 6) 5년(≈5*365일) 기준 확률 계산
total_days_5y = 5 * 365
acc_prob_map = {st: cnt / total_days_5y for st, cnt in acc_map.items()}

# → 이제 acc_prob_map[station] 으로 역별 사고 발생확률을 꺼내 쓸 수 있습니다.


In [44]:
metro_stats

,노선,season,avg_delay,reliability
0,1호선,autumn,2.348315,0.538462
1,1호선,spring,1.892857,0.467391
2,1호선,summer,2.037736,0.728261
3,1호선,winter,2.114286,0.611111
4,2호선,autumn,2.254717,0.340659
5,2호선,spring,1.708333,0.434783
6,2호선,summer,1.548387,0.565217
7,2호선,winter,2.253012,0.422222
8,3호선,autumn,2.007634,0.307692
9,3호선,spring,1.890756,0.380435


In [45]:
acc_prob_map

{'동대문역사문화공원': 0.054246575342465755,
 '사당': 0.03287671232876712,
 '신도림': 0.02684931506849315,
 '고속터미널': 0.025753424657534246,
 '교대': 0.02410958904109589,
 '충무로': 0.023561643835616437,
 '서울': 0.021917808219178082,
 '종로3가': 0.020821917808219178,
 '천호': 0.019726027397260273,
 '군자': 0.019178082191780823,
 '구로디지털단지': 0.01863013698630137,
 '동대문': 0.01808219178082192,
 '잠실': 0.015342465753424657,
 '창동': 0.015342465753424657,
 '합정': 0.015342465753424657,
 '이수': 0.014794520547945205,
 '청량리': 0.014246575342465753,
 '철산': 0.014246575342465753,
 '건대입구': 0.014246575342465753,
 '시청': 0.0136986301369863,
 '노원': 0.0136986301369863,
 '연신내': 0.0136986301369863,
 '신촌': 0.0136986301369863,
 '성신여대입구': 0.01315068493150685,
 '신림': 0.01315068493150685,
 '회현': 0.012602739726027398,
 '대림': 0.012602739726027398,
 '강남': 0.011506849315068493,
 '발산': 0.010958904109589041,
 '을지로3가': 0.010410958904109589,
 '선릉': 0.010410958904109589,
 '성수': 0.010410958904109589,
 '미아사거리': 0.009863013698630137,
 '혜화': 0.009863013698630

In [46]:
# ---------------------------------
# B) 역–노선 매핑 & 최종 점수 계산
# ---------------------------------


# B1) 역 정보 JSON 로드
with open('metro_station.json','r',encoding='utf-8') as f:
    raw = json.load(f)['DATA']
stations_df = pd.DataFrame(raw)
stations_df['station_clean'] = (
    stations_df['name']
    .str.replace(r'\(.*\)$','',regex=True)
    .str.strip()
)
# station_clean ↔ 노선 매핑
station_lines = stations_df[['station_clean','line']].rename(columns={'line':'노선'})

# B2) 노선·계절별 통계(metro_stats) 병합
station_season = pd.merge(
    station_lines,
    metro_stats,      # 앞에서 만든 DataFrame
    on='노선',
    how='left'
)

# B3) 사고확률 매핑 & 최종 점수
# 1) acc_prob_map: 앞서 만든 {station_clean: P(accident)} 맵
station_season['acc_prob'] = (
    station_season['station_clean']
    .map(acc_prob_map)
    .fillna(0.0)
)

# 2) punctuality_score = avg_delay * reliability + accident_probability
station_season['avg_delay']    = station_season['avg_delay'].fillna(0.0)
station_season['reliability'] = station_season['reliability'].fillna(0.0)

station_season['punctuality_score'] = (
    station_season['avg_delay'] * station_season['reliability']
    + station_season['acc_prob']
)

# 결과 확인
print(station_season.head())


  station_clean   노선  season  avg_delay  reliability  acc_prob  \
0           망월사  1호선  autumn   2.348315     0.538462  0.000000   
1           망월사  1호선  spring   1.892857     0.467391  0.000000   
2           망월사  1호선  summer   2.037736     0.728261  0.000000   
3           망월사  1호선  winter   2.114286     0.611111  0.000000   
4           신도림  1호선  autumn   2.348315     0.538462  0.026849   

   punctuality_score  
0           1.264477  
1           0.884705  
2           1.484003  
3           1.292063  
4           1.291326  


In [47]:
# B4) 신대방역 확인
sindaebang = station_season[station_season['station_clean']=='신대방']
sindaebang

,station_clean,노선,season,avg_delay,reliability,acc_prob,punctuality_score
500,신대방,2호선,autumn,2.254717,0.340659,0.006575,0.774666
501,신대방,2호선,spring,1.708333,0.434783,0.006575,0.749329
502,신대방,2호선,summer,1.548387,0.565217,0.006575,0.881751
503,신대방,2호선,winter,2.253012,0.422222,0.006575,0.957847


In [48]:
station_season

,station_clean,노선,season,avg_delay,reliability,acc_prob,punctuality_score
0,망월사,1호선,autumn,2.348315,0.538462,0.000000,1.264477
1,망월사,1호선,spring,1.892857,0.467391,0.000000,0.884705
2,망월사,1호선,summer,2.037736,0.728261,0.000000,1.484003
3,망월사,1호선,winter,2.114286,0.611111,0.000000,1.292063
4,신도림,1호선,autumn,2.348315,0.538462,0.026849,1.291326
...,...,...,...,...,...,...,...
2691,캠퍼스타운,인천1호선,NaN,0.000000,0.000000,0.000000,0.000000
2692,송도달빛축제공원,인천1호선,NaN,0.000000,0.000000,0.000000,0.000000
2693,동막,인천1호선,NaN,0.000000,0.000000,0.000000,0.000000
2694,원인재,인천1호선,NaN,0.000000,0.000000,0.000000,0.000000


In [49]:
# ------------------------------
# 2) 버스 사고 데이터 처리
# ------------------------------
# A) 원본 로드 & 좌표 정제
bus_acc_df = pd.read_excel('bus_accident.xlsx', dtype=str)
# BOM(\ufeff) 제거 후 float 변환
bus_acc_df['좌표X'] = bus_acc_df['좌표X'].str.replace('\ufeff','').astype(float)
bus_acc_df['좌표Y'] = bus_acc_df['좌표Y'].str.replace('\ufeff','').astype(float)

# B) KD‐Tree로 가장 가까운 정류장 인덱스 찾기
coords = bus_acc_df[['좌표Y','좌표X']].to_numpy()   # (N,2) array of (lat,lon)
dists_deg, idxs = bus_tree.query(coords, k=1)      # bus_tree는 정류장 좌표로 만든 cKDTree
bus_acc_df['stop_idx'] = idxs
# degree → meter (대략)
bus_acc_df['dist_m'] = dists_deg * 111_000

# C) 100m 초과 레코드는 드롭
before = len(bus_acc_df)
bus_acc_df = bus_acc_df.loc[bus_acc_df['dist_m'] <= 500.0].copy()
dropped = before - len(bus_acc_df)
print(f">>> {dropped}개의 사고 레코드는 500m 초과로 제거되었습니다.")

# D) 정류장별·노선별 사고 횟수 집계
#    stops_df.index와 stop_idx가 매칭됩니다.
acc_counts = (
    bus_acc_df
      .groupby(['노선명','stop_idx'])
      .size()
      .reset_index(name='acc_count')
)

# E) 5년(≈1825일) 기준 사고 발생 확률 계산
total_days_5y = 5 * 365
acc_counts['acc_prob'] = (acc_counts['acc_count'] / total_days_5y) * 10

# F) 매핑 맵 생성 (key: "노선명_stop_idx" → value: 확률)
acc_prob_map_bus = {
    f"{row['노선명']}_{int(row['stop_idx'])}": row['acc_prob']
    for _, row in acc_counts.iterrows()
}

# 확인
print("\n=== 버스 사고 확률 맵 샘플 ===")
for k,v in list(acc_prob_map_bus.items())[:5]:
    print(k,":",v)

>>> 115개의 사고 레코드는 500m 초과로 제거되었습니다.

=== 버스 사고 확률 맵 샘플 ===
﻿0017_892 : 0.005479452054794521
﻿01A_42 : 0.005479452054794521
﻿01B_581 : 0.005479452054794521
﻿0411_646 : 0.005479452054794521
﻿0411_828 : 0.016438356164383564


In [50]:
# --- 0) valid_pairs 준비 (버스 노드) ---
# node_key = "노선_정류장명" 이므로 split
bus_nodes_df['station'] = (
    bus_nodes_df['node_key']
    .str.split(pat='_', n=1).str[1]
    .str.replace('\ufeff','')  # BOM 제거
    .str.strip()               # 양끝 공백 제거
)
bus_nodes_df['route'] = (
    bus_nodes_df['route']
    .str.replace('\ufeff','')
    .str.strip()
)
valid_pairs = bus_nodes_df[['route','station']].drop_duplicates()


# --- 1) bus_late.xlsx 로드 & BOM/공백 제거 ---
late_df = pd.read_excel('bus_late.xlsx', dtype=str)
# 컬럼명에도 BOM이 껴 있을 수 있으니 실제 컬럼명 확인한 뒤
# 아래 예에서는 '노선명','정류장명' 컬럼에 적용
for c in ['노선명','정류장명']:
    late_df[c] = (
        late_df[c]
        .str.replace('\ufeff','')
        .str.strip()
    )

# --- 2) 매핑 불가 조합 날리기 (inner merge) ---
before = len(late_df)
late_df = late_df.merge(
    valid_pairs,
    how='inner',
    left_on = ['노선명','정류장명'],
    right_on= ['route','station']
)
print(f">>> {before - len(late_df)}개 레코드가 매핑 불가로 제거되었습니다.")
# 이제 late_df 안에는 분명 “472”-“순천향대학병원” 조합이 남아 있을 거예요.


# --- 3) 그룹별 통계 집계 & max_delay 계산 ---
# (BOM 제거·타입 변환은 이전과 동일)
for col in ['평균2분이하건수','평균2분초과건수']:
    late_df[col] = (
        late_df[col].str.replace('\ufeff','')
                   .astype(float)
    )
# season 컬럼 제거
if 'season' in late_df.columns:
    late_df = late_df.drop(columns='season')


bus_late_stats = (
    late_df
    .groupby(['정류장명','노선명'], as_index=False)
    .agg(
        avg_under2=('평균2분이하건수','mean'),
        avg_over2 =('평균2분초과건수','mean'),
    )
)
bus_late_stats['total_cnt'] = bus_late_stats['avg_under2'] + bus_late_stats['avg_over2']
bus_late_stats['over2_pct'] = (bus_late_stats['avg_over2'] / bus_late_stats['total_cnt'])

def assign_max_delay(row):
    u, o = row['avg_under2'], row['avg_over2']
    if   o <=   u: return  2
    elif o <= 2*u: return  2.3
    elif o <= 3*u: return 2.7
    else:           return 3

bus_late_stats['max_delay'] = bus_late_stats.apply(assign_max_delay, axis=1)

print(bus_late_stats.head())


>>> 131634개 레코드가 매핑 불가로 제거되었습니다.
              정류장명   노선명   avg_under2   avg_over2    total_cnt  over2_pct  \
0           (구)법화사  성북02  1639.500000  205.166667  1844.666667   0.111222   
1  123전자타운.2001아울렛  구로05  2507.500000  764.000000  3271.500000   0.233532   
2  123전자타운.2001아울렛  구로06  2052.666667  644.000000  2696.666667   0.238813   
3           14단지상가  노원02  2993.166667  277.666667  3270.833333   0.084892   
4           14단지상가  노원11  2373.333333  273.166667  2646.500000   0.103218   

   max_delay  
0        2.0  
1        2.0  
2        2.0  
3        2.0  
4        2.0  


In [51]:
# max_delay 분포 계산
delay_dist = (
    bus_late_stats['max_delay']
    .value_counts()
    .sort_index()              # 2, 5, 10, 15 순으로 정렬
    .rename_axis('max_delay')  # 인덱스 이름 지정
    .reset_index(name='count') # 컬럼명 지정
)

print(delay_dist)


   max_delay  count
0        2.0  25080
1        2.3    108
2        2.7     31
3        3.0     22


In [52]:
# ---------------------------------
# B) 버스 노드–정류장 매핑 & 최종 점수 계산
# ---------------------------------
import pandas as pd

# 1) node_key에서 '정류장명' 추출 & 컬럼 정리
bus_nodes_df['정류장명'] = bus_nodes_df['node_key'].str.split('_', n=1).str[1]
bus_nodes_df['노선명']  = bus_nodes_df['route']  # route 컬럼이 노선명

nodes = bus_nodes_df[['node_key','stop_idx','노선명','정류장명']]

# 2) bus_late_stats 를 '노선명','정류장명' 기준으로만 집계
# (season 컬럼을 완전히 제거하고, 전체 데이터 합산 혹은 평균을 내두었다고 가정)
# bus_late_stats: ['정류장명','노선명','avg_under2','avg_over2','total_cnt','over2_pct','max_delay']
bus_late_agg = bus_late_stats.drop(columns=['season'], errors='ignore').drop_duplicates(
    subset=['정류장명','노선명']
)

# 3) 노드 정보에 버스 지연 통계 병합
node_season_bus = pd.merge(
    nodes,
    bus_late_agg,
    on=['노선명','정류장명'],
    how='left'
)

# 4) 사고 확률 매핑
node_season_bus['acc_prob'] = node_season_bus.apply(
    lambda row: acc_prob_map_bus.get(f"{row['노선명']}_{int(row['stop_idx'])}", 0.0),
    axis=1
)

# 5) 결측치 처리
node_season_bus['max_delay']      = node_season_bus['max_delay'].fillna(0.0)
node_season_bus['total_cnt']      = node_season_bus['total_cnt'].fillna(1.0)  # 분모 0 방지
node_season_bus['avg_under2']     = node_season_bus['avg_under2'].fillna(0.0)
node_season_bus['avg_over2']      = node_season_bus['avg_over2'].fillna(0.0)
node_season_bus['over2_pct']      = node_season_bus['over2_pct'].fillna(0.0)

# on-time 비율 (2분 이하 비율)
node_season_bus['reliability_bus'] = (
    node_season_bus['avg_under2'] / node_season_bus['total_cnt']
)

# 6) 최종 punctuality_score 계산
node_season_bus['punctuality_score'] = (
    node_season_bus['max_delay'] * node_season_bus['reliability_bus']
    + node_season_bus['acc_prob']
)

# 7) 결과 예시 출력 (season 컬럼 제거)
print(node_season_bus[[
    'node_key','노선명','정류장명',
    'max_delay','reliability_bus','acc_prob','punctuality_score'
]].head())


        node_key   노선명      정류장명  max_delay  reliability_bus  acc_prob  \
0   0017_신용산지하차도  0017   신용산지하차도        2.0         0.849049       0.0   
1       0017_용산역  0017       용산역        2.0         0.794074       0.0   
2  0017_용산푸르지오써밋  0017  용산푸르지오써밋        2.0         0.812369       0.0   
3    0017_한강대교북단  0017    한강대교북단        2.0         0.782426       0.0   
4   0017_서부이촌동입구  0017   서부이촌동입구        2.0         0.839126       0.0   

   punctuality_score  
0           1.698099  
1           1.588148  
2           1.624738  
3           1.564853  
4           1.678252  


In [53]:
# station_season: ['station_clean','노선','punctuality_score'] 컬럼을 가진 DataFrame
for _, row in station_season.iterrows():
    key = f"{row['노선']}_{row['station_clean']}"   # 예: '2호선_잠실'
    if key in integrated_graph:
        integrated_graph.nodes[key]['punctuality_score'] = row['punctuality_score']


In [54]:
# node_season_bus: ['node_key','punctuality_score'] 컬럼을 가진 DataFrame
for _, row in node_season_bus.iterrows():
    key = row['node_key']  # 예: '140_순천향대학병원'
    if key in integrated_graph:
        integrated_graph.nodes[key]['punctuality_score'] = row['punctuality_score']


In [55]:
for n, data in integrated_graph.nodes(data=True):
    data.setdefault('punctuality_score', 0.0)


In [56]:
def get_node_pleasantness(node: str, G: nx.Graph) -> float:
    """
    통합 그래프 G의 node 에 붙어 있는 'punctuality_score'를 꺼내옵니다.
    (Metro, Bus 노드 모두 동일하게 사용)
    """
    return G.nodes[node].get('punctuality_score', 0.0)


In [57]:
def evaluate_pleasantness_for_candidates(
    G: nx.Graph,
    candidate_routes: list[dict],
    k: int = 10
) -> list[dict]:
    """
    candidate_routes: get_fastest_routes 등으로 구한 경로 리스트.
                      각 원소는 최소한 'path', 'transfers', 'total_time_min' 키를 가집니다.
    반환: 각 경로에 'pleasantness_score'를 추가하고,
         낮은 점수(더 쾌적) 순으로 상위 k개를 반환합니다.
    """
    results = []
    for r in candidate_routes:
        path = r['path']
        # 1) 각 노드의 punctuality_score 합산
        total = sum(G.nodes[n].get('punctuality_score', 0.0) for n in path)
        n_nodes = len(path)
        # 2) 평균 계산
        pleasant_avg = total / n_nodes if n_nodes > 0 else 0.0
        # 3) 복합 스코어: 평균*0.9 + 노드 개수*0.1
        pleasant_score = pleasant_avg * 0.9 + n_nodes * 0.1

        entry = {
            **r,
            'pleasantness_score': pleasant_score
        }
        results.append(entry)

    # 4) 정렬: 복합 스코어 ↑(낮은 게 우선), 동률이면 환승↑, 소요시간↑
    results.sort(key=lambda x: (
        x['pleasantness_score'],
        x['transfers'],
        x.get('time_min', x.get('total_time_min', 0))
    ))
    return results[:k]


In [58]:
# 2) 최단 경로(k=50) 탐색 — transit_graph 는 metro+bus 통합 그래프
#    get_fastest_routes 함수는 예시입니다. 실제 이름/인자는 사용 중인 구현에 맞춰 바꿔주세요.
candidate_routes = get_fastest_routes_optimized(
    integrated_graph,
    start_candidates,
    end_candidates,
    k=50,
    max_transfers=4
)

# 3) 쾌적도(정시성) 평가
top_pleasant = evaluate_pleasantness_for_candidates(
    integrated_graph,
    candidate_routes,
    k=10
)

# 4) 결과 출력
for i, r in enumerate(top_pleasant, 1):
    print(
        f"{i}위 (정시성 합계 {r['pleasantness_score']:.3f}, "
        f"{r['time_min']:.1f}분, 환승 {r['transfers']}회): {r['path']}"
    )

1위 (정시성 합계 1.598, 46.7분, 환승 1회): ['3호선_을지로3가', '3호선_충무로', '3호선_동대입구', '3호선_약수', '3호선_금호', '3호선_옥수', '3호선_압구정', '3호선_신사', '신분당선_신사', '신분당선_논현', '신분당선_신논현']
2위 (정시성 합계 1.686, 38.3분, 환승 1회): ['3호선_을지로3가', '3호선_충무로', '3호선_동대입구', '3호선_약수', '3호선_금호', '3호선_옥수', '3호선_압구정', '3호선_신사', '신분당선_신사', '신분당선_논현', '신분당선_신논현', '신분당선_강남']
3위 (정시성 합계 1.704, 36.8분, 환승 1회): ['3호선_종로3가', '3호선_을지로3가', '3호선_충무로', '3호선_동대입구', '3호선_약수', '3호선_금호', '3호선_옥수', '3호선_압구정', '3호선_신사', '신분당선_신사', '신분당선_논현', '신분당선_신논현']
4위 (정시성 합계 1.792, 28.4분, 환승 1회): ['3호선_종로3가', '3호선_을지로3가', '3호선_충무로', '3호선_동대입구', '3호선_약수', '3호선_금호', '3호선_옥수', '3호선_압구정', '3호선_신사', '신분당선_신사', '신분당선_논현', '신분당선_신논현', '신분당선_강남']
5위 (정시성 합계 1.834, 52.1분, 환승 2회): ['2호선_을지로4가', '2호선_동대문역사문화공원', '2호선_신당', '2호선_상왕십리', '2호선_왕십리', '수인분당선_왕십리', '수인분당선_서울숲', '수인분당선_압구정로데오', '수인분당선_강남구청', '수인분당선_선정릉', '9호선_선정릉', '9호선_언주', '9호선_신논현']
6위 (정시성 합계 1.855, 50.2분, 환승 2회): ['2호선_을지로4가', '2호선_을지로3가', '3호선_을지로3가', '3호선_충무로', '3호선_동대입구', '3호선_약수', '3호선_금호', '3호선_옥수', '3호선_압구정',

# 쾌적도 함수

In [73]:
# 0. Colab 파일 업로드 (한 번만 필요)
uploaded = files.upload()

Saving weather.xlsx to weather.xlsx
Saving metro_dust.xlsx to metro_dust (1).xlsx


In [75]:
import pandas as pd
import numpy as np
from sklearn.neighbors import BallTree

# 1. 데이터 로드
df_metro = pd.read_excel('metro_dust.xlsx')
df_weather = pd.read_excel('weather.xlsx')

In [77]:
import pandas as pd
import numpy as np
from sklearn.neighbors import BallTree

# 1. 데이터 로드
df_metro = pd.read_excel('metro_dust.xlsx')
df_weather = pd.read_excel('weather.xlsx')
# 버스 정거장 데이터 로드 (필요에 맞게 파일 경로 수정)
df_bus = pd.read_excel('bus_stops.xlsx')

# 2. 컬럼 확인
print("Metro columns:", df_metro.columns.tolist())
print("Weather columns:", df_weather.columns.tolist())
print("Bus columns:", df_bus.columns.tolist())

# 3. 컬럼명 자동 감지
metro_cols   = df_metro.columns.tolist()
weather_cols = df_weather.columns.tolist()
bus_cols     = df_bus.columns.tolist()

# Metro
metro_line_col     = [c for c in metro_cols   if '호선' in c or 'line' in c.lower()][0]
metro_station_col  = [c for c in metro_cols   if '역' in c or 'station' in c.lower()][0]
metro_lat_col      = [c for c in metro_cols   if '위도' in c or 'lat' in c.lower()][0]
metro_lon_col      = [c for c in metro_cols   if '경도' in c or 'lon' in c.lower()][0]
metro_dust_col     = [c for c in metro_cols   if '먼지' in c or 'dust' in c.lower()][0]

# Weather
weather_lat_col    = [c for c in weather_cols if '위도' in c or 'lat' in c.lower()][0]
weather_lon_col    = [c for c in weather_cols if '경도' in c or 'lon' in c.lower()][0]
weather_temp_col   = [c for c in weather_cols if '기온' in c or 'temp' in c.lower()][0]
weather_hum_col    = [c for c in weather_cols if '습도' in c or 'hum' in c.lower()][0]
weather_dust_col   = [c for c in weather_cols if '먼지' in c or 'dust' in c.lower()][0]

# Bus
bus_route_col      = [c for c in bus_cols     if '번호' in c or 'route' in c.lower()][0]
bus_stop_col       = [c for c in bus_cols     if '정거장' in c or 'stop' in c.lower() or '역' in c][0]
bus_lat_col        = [c for c in bus_cols     if '위도' in c or 'lat' in c.lower()][0]
bus_lon_col        = [c for c in bus_cols     if '경도' in c or 'lon' in c.lower()][0]

# 4. BallTree 준비 (단위: 라디안, 거리는 지구 반지름 6371km 사용)
coords_weather = np.deg2rad(df_weather[[weather_lat_col, weather_lon_col]].values)
tree = BallTree(coords_weather, metric='haversine')

def map_temp_hum(lat, lon):
    dist, idx = tree.query(np.deg2rad([[lat, lon]]), k=1)
    rec = df_weather.iloc[idx[0][0]]
    return rec[weather_temp_col], rec[weather_hum_col]

def map_all(lat, lon):
    dist, idx = tree.query(np.deg2rad([[lat, lon]]), k=1)
    rec = df_weather.iloc[idx[0][0]]
    return rec[weather_temp_col], rec[weather_hum_col], rec[weather_dust_col]

# 5. 지하철 매핑: 미세먼지(df_metro 내), 기온·습도(weather에서)
df_metro[['temperature', 'humidity']] = df_metro.apply(
    lambda r: map_temp_hum(r[metro_lat_col], r[metro_lon_col]),
    axis=1, result_type='expand'
)

# 6. 버스 매핑: 기온·습도·미세먼지 모두(weather에서)
df_bus[['temperature', 'humidity', 'dust']] = df_bus.apply(
    lambda r: map_all(r[bus_lat_col], r[bus_lon_col]),
    axis=1, result_type='expand'
)

# 7. 결과 확인
print("\n=== Metro Mapped ===")
print(df_metro.head())

print("\n=== Bus Mapped ===")
print(df_bus.head())


Metro columns: ['지점명', '호선', '위치', '월평균', '계절']
Weather columns: ['지역', 'season', 'avg_humidity', 'avg_temp', 'lng', 'lat', '평균미세먼지']
Bus columns: ['NODE_ID', 'ARS_ID', '정류소명', 'X좌표', 'Y좌표', '정류소타입']


IndexError: list index out of range

In [62]:
bus_dust

,지역,계절,평균미세먼지
0,강남구,fall,29.018442
1,강남구,spring,37.697315
2,강남구,summer,23.133761
3,강남구,winter,38.786049
4,강동구,fall,28.931552
...,...,...,...
95,중구,winter,39.207009
96,중랑구,fall,27.460435
97,중랑구,spring,38.902372
98,중랑구,summer,23.736022


In [63]:
metro_dust

,지점명,호선,위치,월평균,계절
0,청량리,1호선,서울,43.1,winter
1,제기동,1호선,서울,66.5,winter
2,신설동,1호선,서울,64.7,winter
3,동묘,1호선,서울,70.7,winter
4,동대문,1호선,서울,84.7,winter
...,...,...,...,...,...
3978,신림,신림선,서울,20.2,winter
3979,서원,신림선,서울,17.7,winter
3980,서울대벤처타운,신림선,서울,15.7,winter
3981,관악산(서울대),신림선,서울,13.7,winter


In [64]:
weather_water

,자치구,계절,온도 최대(℃),온도 평균(℃),온도 최소(℃),습도 최대(%),습도 평균(%),습도 최소(%)
0,강남구,fall,18.68,18.28,17.90,71.22,69.62,67.95
1,강남구,spring,14.70,14.22,13.77,56.59,54.91,53.13
2,강남구,summer,28.65,28.23,27.84,78.92,77.45,75.92
3,강남구,winter,1.45,1.08,0.73,55.07,53.46,51.78
4,강동구,fall,18.04,17.60,17.19,75.33,73.71,72.02
...,...,...,...,...,...,...,...,...
95,중구,winter,1.59,1.23,0.89,53.22,51.68,50.03
96,중랑구,fall,18.85,18.40,17.97,68.07,66.53,64.92
97,중랑구,spring,15.07,14.56,14.08,56.54,54.90,53.20
98,중랑구,summer,28.92,28.46,28.02,75.47,73.82,72.14


In [68]:
from datetime import datetime
import pandas as pd

# 1) 계절 판별 함수 (기존과 동일)
def classify_season(dt: datetime) -> str:
    m = dt.month
    if 3 <= m <= 5:
        return 'spring'
    elif 6 <= m <= 8:
        return 'summer'
    elif 9 <= m <= 11:
        return 'fall'
    else:
        return 'winter'

# 2) 지하철역 미세먼지 가져오는 함수(수정본)
def get_subway_dust(df_metro: pd.DataFrame,
                    line: str,
                    station: str,
                    dt: datetime = None) -> float | None:
    """
    df_metro: metro_dust DataFrame (컬럼: '호선', '지점명', '계절', '월평균')
    line   : ex) '1호선'
    station: ex) '청량리'
    dt     : 조회할 datetime (None이면 현재 시간)
    """
    if dt is None:
        dt = datetime.now()
    season = classify_season(dt)

    # 필터링
    mask = (
        (df_metro['호선']   == line) &
        (df_metro['지점명'] == station) &
        (df_metro['계절']   == season)
    )
    subset = df_metro.loc[mask, '월평균']
    if subset.empty:
        return None

    return float(subset.mean())

# 3) 사용 예시
from datetime import datetime
test_dt = datetime(2025, 7, 15, 14, 0)   # 여름
val = get_subway_dust(metro_dust, '1호선', '청량리', test_dt)
print(f"{test_dt} 기준 1호선 청량리역 월평균 미세먼지: {val} µg/m³")


2025-07-15 14:00:00 기준 1호선 청량리역 월평균 미세먼지: 28.166666666666668 µg/m³


In [72]:
from datetime import datetime
import pandas as pd
from geopy.distance import geodesic

# 1) 계절 판별 함수 (기존 그대로)
def classify_season(dt: datetime) -> str:
    m = dt.month
    if 3 <= m <= 5:
        return 'spring'
    elif 6 <= m <= 8:
        return 'summer'
    elif 9 <= m <= 11:
        return 'fall'
    else:
        return 'winter'

# 2) 계절별 가장 가까운 관측소 날씨(기온·습도) 함수
def get_station_weather(lat: float,
                        lng: float,
                        dt: datetime = None) -> tuple[float, float] | None:
    """
    weather_seasons DataFrame 사용
    컬럼: 'nam'(자치구명), 'lat', 'lng', '계절',
         '온도 평균(℃)', '습도 평균(%)' 등
    """
    if dt is None:
        dt = datetime.now()
    season = classify_season(dt)

    # weather_seasons에서 해당 계절만 추출
    df_s = (
        weather_seasons[weather_seasons['season'] == season]
        .dropna(subset=['lat','lng','온도 평균(℃)','습도 평균(%)'])
        .assign(
            lat=lambda d: pd.to_numeric(d['lat'], errors='coerce'),
            lng=lambda d: pd.to_numeric(d['lng'], errors='coerce')
        )
    )
    if df_s.empty:
        return None

    # 각 관측소까지 거리 계산
    df_s['dist'] = df_s.apply(
        lambda r: geodesic((lat, lng), (r['lat'], r['lng'])).meters,
        axis=1
    )
    nearest = df_s.loc[df_s['dist'].idxmin()]

    return float(nearest['온도 평균(℃)']), float(nearest['습도 평균(%)'])

# 3) 사용 예시
# 예: 2025-08-01 10시, 위도·경도 (37.5796, 127.0094)인 지점
from datetime import datetime
test_dt = datetime(2025, 8, 1, 10, 0)
temp, hum = get_station_weather(37.5796, 127.0094, test_dt)
print(f"{test_dt} 기준 → 기온: {temp}℃, 습도: {hum}%")


KeyError: ['온도 평균(℃)', '습도 평균(%)']

# 혼잡도 함수

In [63]:
uploaded = files.upload()

Saving bus_confuse_speed.xlsx to bus_confuse_speed.xlsx
Saving metro_confuse.xlsx to metro_confuse.xlsx
Saving bus_remaining.csv to bus_remaining.csv


In [64]:
# 0) 데이터 불러오기
df_metro_confuse = pd.read_excel('metro_confuse.xlsx', engine='openpyxl')
df_bus_confuse = pd.read_csv('bus_remaining.csv', encoding='utf-8')  # cp949 필요 시 변경
df_bus_speed = pd.read_excel('bus_confuse_speed.xlsx', engine='openpyxl')


<ipython-input-64-2654779954>:3: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_bus_confuse = pd.read_csv('bus_remaining.csv', encoding='utf-8')  # cp949 필요 시 변경


In [67]:
df_bus_speed

,구분별(1),구분별(2),2024. 01,2024. 02,2024. 03,2024. 04,2024. 05,2024. 06,2024. 07,2024. 08,2024. 09,2024. 10,2024. 11,2024. 12
0,평균,소계,23.2,22.9,23.0,22.8,22.7,22.8,22.5,22.3,22.4,22.5,22.3,22.7
1,권역별,강남,23.7,23.4,23.6,23.3,23.3,23.3,22.9,22.7,23.0,23.1,22.7,23.1
2,NaN,강북,22.7,22.3,22.4,22.3,22.2,22.3,22.0,21.9,21.8,22.0,21.9,22.4
3,자치구별,종로구,22.4,22.1,22.2,21.7,21.7,21.9,21.7,21.5,21.3,21.3,21.3,21.8
4,NaN,중구,20.3,20.2,20.0,19.6,19.7,19.9,19.6,19.6,19.7,19.3,19.2,19.7
5,NaN,용산구,26.4,26.1,26.2,25.7,25.7,25.7,25.4,25.2,25.4,25.4,25.2,25.7
6,NaN,성동구,24.3,23.9,24.2,23.9,23.9,24.0,23.5,23.5,23.4,23.6,23.3,23.9
7,NaN,광진구,23.8,23.4,23.5,23.2,23.0,23.3,22.9,22.9,22.9,23.2,23.0,23.6
8,NaN,동대문구,22.7,22.5,22.7,22.6,22.5,22.6,22.4,22.3,22.1,22.4,22.2,22.8
9,NaN,중랑구,22.0,21.6,21.7,21.6,21.3,21.5,21.4,21.3,21.1,21.4,21.4,21.9


In [66]:
df_bus_confuse

,노선번호,노선명,표준버스정류장ID,버스정류장ARS번호,역명,season,00시잔여승객수,1시잔여승객수,2시잔여승객수,3시잔여승객수,...,14시잔여승객수,15시잔여승객수,16시잔여승객수,17시잔여승객수,18시잔여승객수,19시잔여승객수,20시잔여승객수,21시잔여승객수,22시잔여승객수,23시잔여승객수
0,470,470번(상암차고지~안골마을),100000001,1001,종로2가사거리(00066),winter,0.0,0.0,0.0,0.000000,...,580.0,478.5,363.5,187.5,0.0,0.0,40.0,110.5,143.500000,201.000000
1,N37,N37번(진관공영차고지~송파공영차고지),100000001,1001,종로2가사거리(00089),winter,0.0,0.0,38.0,147.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,215.333333,215.333333
2,470,470번(상암차고지~안골마을),100000001,1001,종로2가사거리(00064),winter,0.0,0.0,0.0,0.000000,...,786.0,692.0,550.0,421.0,237.0,154.0,242.0,369.0,433.000000,511.000000
3,741,741번(진관차고지~헌인릉입구),100000001,1001,종로2가사거리(00073),winter,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,65.0,157.0,178.000000,210.000000
4,N37,N37번(송파공영차고지~진관공영차고지),100000001,1001,종로2가사거리(00032),winter,0.0,0.0,29.0,30.666667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.666667,30.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180887,3323,3323번(강동차고지~중앙보훈병원역),227000488,28100,미사강변더샵센트럴포레.동원로얄듀크(00008),summer,0.0,0.0,0.0,0.000000,...,458.0,506.0,655.0,876.0,1295.0,1394.0,1473.0,1503.0,1542.000000,1547.000000
180888,3323,3323번(강동차고지~중앙보훈병원역),227000632,28439,미사강변더샵센트럴포레.동원로얄듀크(00039),summer,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
180889,101,101번(화계사~동대문),998501983,~,동아운수종점(종점가상)(00084),summer,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
180890,6633,6633번(강서공영차고지~여의도역),998502032,~,강서공영차고지(종점가상)(00100),summer,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000


In [65]:
df_metro_confuse

,요일구분,호선,역번호,출발역,상하구분,5시30분,6시00분,6시30분,7시00분,7시30분,...,20시00분,20시30분,21시00분,21시30분,22시00분,22시30분,23시00분,23시30분,00시00분,00시30분
0,평일,1,158,청량리,상선,7.2,6.9,4.5,8.3,10.4,...,24.8,26.1,28.2,24.5,23.0,22.2,21.7,14.9,8.5,0.0
1,평일,1,157,제기동,상선,7.6,8.7,6.5,8.7,12.9,...,30.0,26.0,34.8,27.5,25.7,25.4,24.2,16.8,11.6,0.0
2,평일,1,156,신설동,상선,6.7,11.2,7.2,9.6,14.8,...,30.7,26.8,36.3,28.6,26.6,26.1,25.2,16.1,12.6,0.0
3,평일,1,159,동묘앞,상선,6.3,11.8,7.4,12.2,17.7,...,32.1,30.1,41.8,29.9,29.0,23.5,27.7,13.5,14.3,0.0
4,평일,1,155,동대문,상선,7.4,11.2,8.3,14.0,21.7,...,35.6,33.5,40.5,34.7,32.6,26.1,31.3,18.0,13.6,3.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,평일,우이신설선,4702,솔밭공원,하선,22.4,19.4,15.9,20.5,33.5,...,30.3,28.5,41.3,36.3,26.8,16.3,21.6,18.8,10.0,6.5
2996,토요일,우이신설선,4702,솔밭공원,상선,15.0,17.4,7.3,22.4,25.8,...,36.6,29.2,27.2,27.7,38.3,25.5,17.5,18.8,5.7,3.1
2997,토요일,우이신설선,4702,솔밭공원,하선,21.6,14.7,13.4,21.1,26.2,...,25.6,29.4,28.7,29.2,29.2,32.5,7.9,13.8,5.2,4.8
2998,일요일,우이신설선,4702,솔밭공원,상선,7.0,17.2,19.3,21.8,37.0,...,28.3,31.6,27.0,21.2,27.1,24.0,25.9,7.7,2.8,10.7


In [68]:
from zoneinfo import ZoneInfo

# 1) 요일구분 컬럼 정제
df_metro_confuse['요일구분'] = df_metro_confuse['요일구분'].str.strip()

# 2) 시간별 승객 수 컬럼 식별 (HH:MM 형식)
time_cols = [col for col in df_metro_confuse.columns if isinstance(col, str) and ':' in col]

# 3) 지하철 최대 수용인원(34명)으로 나누어 비율화
df_metro_confuse[time_cols] = df_metro_confuse[time_cols].div(34.0)

# 4) 시간 문자열 → time 객체 매핑
from datetime import datetime as _dt
time_map = {
    _dt.strptime(col, '%H:%M').time(): col
    for col in time_cols
}

def get_metro_congestion(line: str, station: str, now: datetime = None) -> float:
    """
    line: '2호선' 등
    station: '청량리' 등
    now: datetime 객체 (Asia/Seoul). None이면 현재 시간 사용.
    반환: 해당 시점의 혼잡 비율 (0.0~1.0 이상)
    """
    if now is None:
        now = datetime.now(ZoneInfo('Asia/Seoul'))
    wd = now.weekday()
    if wd < 5:
        day_type = '평일'
    elif wd == 5:
        day_type = '토요일'
    else:
        day_type = '일요일'

    # 필터링
    df = df_metro_confuse[
        (df_metro_confuse['요일구분'] == day_type) &
        (df_metro_confuse['호선'] == line) &
        (df_metro_confuse['출발역'] == station)
    ]
    if df.empty:
        return 0.0

    row = df.iloc[0]
    current_time = now.time()
    # 가장 가까운 시간 컬럼 찾기
    nearest_time = min(
        time_map.keys(),
        key=lambda t: abs((datetime.combine(date.today(), t) - datetime.combine(date.today(), current_time)).total_seconds())
    )
    col = time_map[nearest_time]
    return float(row[col])

# 사용 예시
print("평일 08:15 기준 2호선 청량리 혼잡도 비율:",
      get_metro_congestion('2호선', '청량리', datetime(2025, 6, 16, 8, 15)))

평일 08:15 기준 2호선 청량리 혼잡도 비율: 0.0


22.7

In [71]:
# 1) 컬럼명 깔끔히 바꾸기
df_bus_speed = df_bus_speed.rename(
    columns={
        '구분별(1)': 'category',
        '구분별(2)': 'region'
    }
)

# 2) 불필요한 (요구하신) 두 행 제거
#    — category가 '권역별'이거나, region이 NaN인 요약 행을 삭제
df_bus_speed = df_bus_speed[~(
    (df_bus_speed['category'] == '권역별') |
    (df_bus_speed['region'].isna())
)].reset_index(drop=True)

# 3) 월별 컬럼 이름들 확인 (공백까지 정확히 맞춰주세요)
months = ['2024. 01','2024. 02','2024. 03','2024. 04','2024. 05','2024. 06',
          '2024. 07','2024. 08','2024. 09','2024. 10','2024. 11','2024. 12']

# 4) 계절별 평균 계산
season_months = {
    'spring': months[2:5],   # 3,4,5월
    'summer': months[5:8],   # 6,7,8월
    'fall':   months[8:11],  # 9,10,11월
    'winter': [months[11], months[0], months[1]]  # 12,1,2월
}

# 새 DataFrame 준비: category, region 유지
bus_speed_season = df_bus_speed[['category','region']].copy()

# 각 계절별 평균 내기
for season, mths in season_months.items():
    bus_speed_season[season] = (
        df_bus_speed[mths]
        .astype(float)
        .mean(axis=1)
    )

# 5) 22.7로 나누어 비율화
bus_speed_season[['spring','summer','fall','winter']] /= 22.7

# 6) 결과 확인
print(bus_speed_season.head())

  category region    spring    summer      fall    winter
0       평균     소계  1.005874  0.992658  0.986784  1.010279
1     자치구별    종로구  0.963289  0.955947  0.938326  0.973568
2      NaN     중구  0.870778  0.867841  0.854626  0.883994
3      NaN    용산구  1.139501  1.120411  1.116006  1.148311
4      NaN    성동구  1.057269  1.042584  1.032305  1.058737


In [72]:
bus_speed_season

,category,region,spring,summer,fall,winter
0,평균,소계,1.005874,0.992658,0.986784,1.010279
1,자치구별,종로구,0.963289,0.955947,0.938326,0.973568
2,NaN,중구,0.870778,0.867841,0.854626,0.883994
3,NaN,용산구,1.139501,1.120411,1.116006,1.148311
4,NaN,성동구,1.057269,1.042584,1.032305,1.058737
5,NaN,광진구,1.023495,1.014684,1.014684,1.039648
6,NaN,동대문구,0.995595,0.988253,0.979442,0.998532
7,NaN,중랑구,0.948605,0.942731,0.938326,0.961821
8,NaN,성북구,1.032305,1.016153,1.004405,1.032305
9,NaN,강북구,0.886931,0.879589,0.873715,0.885463


In [85]:
# 1) '노선명' 컬럼 제거
df_bus_confuse = df_bus_confuse.drop(columns=['노선명'], errors='ignore')

# 2) 그룹키 및 숫자형 컬럼 지정
group_cols = ['노선번호', 'season', '역명']
# '잔여승객수' 포함 컬럼만 선택
numeric_cols = [c for c in df_bus_confuse.columns if '잔여승객수' in c]

# 3) 그룹별 평균 계산
bus_confuse_grouped = (
    df_bus_confuse
      .groupby(group_cols)[numeric_cols]
      .mean()
      .reset_index()
)

# 4) 1시간당 10대 → 10로 나누기, 4계절
bus_confuse_grouped[numeric_cols] /= 40.0

# 5) 좌석 20개 가정 → 26으로 나누기
bus_confuse_grouped[numeric_cols] /= 26.0

# 6) 시간 컬럼 파싱 (HH시잔여승객수 → time 객체 매핑)
time_map = {}
for col in numeric_cols:
    hour = int(col.split('시')[0])
    time_map[datetime.strptime(f"{hour:02d}:00", "%H:%M").time()] = col

# 7) 혼잡도 조회 함수 정의
def get_bus_congestion(route, station, now=None):
    """
    route: 버스번호 (문자열 또는 숫자)
    station: 역명 (문자열)
    now: datetime (Asia/Seoul), None이면 현재시각
    반환: 잔여 승객 비율 (0.0~1.0)
    """
    if now is None:
        now = datetime.now(ZoneInfo("Asia/Seoul"))
    season = get_current_season(now)
    df = bus_confuse_grouped[
        (bus_confuse_grouped['노선번호'].astype(str) == str(route)) &
        (bus_confuse_grouped['season'] == season) &
        (bus_confuse_grouped['역명'] == station)
    ]
    if df.empty:
        return 0.0
    row = df.iloc[0]
    current_time = now.time().replace(minute=0, second=0, microsecond=0)
    # 가장 가까운 시간 찾기
    nearest = min(time_map.keys(),
                  key=lambda t: abs((datetime.combine(date.today(), t) - datetime.combine(date.today(), current_time)).total_seconds()))
    return float(row[time_map[nearest]])



In [86]:
from datetime import datetime, date
from zoneinfo import ZoneInfo

# 계절 계산 함수 정의 (예전 정의 재사용)
def get_current_season(now=None):
    if now is None:
        now = datetime.now(ZoneInfo("Asia/Seoul"))
    m = now.month
    if 3 <= m <= 5:
        return 'spring'
    if 6 <= m <= 8:
        return 'summer'
    if 9 <= m <= 11:
        return 'autumn'
    return 'winter'

# 버스 혼잡도 예시 출력 (여름 평일 08:15 기준)
print(
    "여름 평일 08:15 기준 01A 노선 KT광화문지사(00016) 혼잡도 비율:",
    get_bus_congestion('01A', 'KT광화문지사(00016)', datetime(2025, 6, 16, 19, 15))
)

여름 평일 08:15 기준 01A 노선 KT광화문지사(00016) 혼잡도 비율: 5.334615384615384


In [93]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from zoneinfo import ZoneInfo
from datetime import datetime

# 1) 지오코더 초기화 (한글 주소 요청)
geolocator = Nominatim(user_agent="station_region_mapper")
reverse = RateLimiter(geolocator.reverse, min_delay_seconds=1, max_retries=2)

def get_district(lat, lon):
    """
    위경도 → ‘구’ 이름 반환. 실패 시 None.
    """
    loc = reverse((lat, lon), language='ko')
    if not loc or 'address' not in loc.raw:
        return None
    addr = loc.raw['address']
    # 여러 키 중에 ‘구’를 뜻하는 항목을 골라서 반환
    for key in ('suburb','city_district','county'):
        if key in addr and addr[key].endswith('구'):
            return addr[key]
    return None

# 2) 각 버스/지하철 노드에 region 정보 부착
for node, data in integrated_graph.nodes(data=True):
    if data.get('mode') in ('bus','metro'):
        lat, lon = data.get('lat'), data.get('lng')
        region = get_district(lat, lon)
        integrated_graph.nodes[node]['region'] = region or 'Unknown'

# 이렇게 만든 region 정보로 앞서 만든 region_speed_map 과 매핑해서 사용하세요.


KeyboardInterrupt: 

In [94]:
import geopandas as gpd
from shapely.geometry import Point

# (A) 경계 파일 읽기
districts = gpd.read_file('seoul_districts.geojson').to_crs('EPSG:4326')

# (B) 정류장 DataFrame → GeoDataFrame
stops_df = pd.read_excel('bus_stops_seoul.xlsx')
stops_gdf = gpd.GeoDataFrame(
    stops_df,
    geometry=[Point(xy) for xy in zip(stops_df['X좌표'], stops_df['Y좌표'])],
    crs='EPSG:4326'
)

# (C) 공간 결합
joined = gpd.sjoin(stops_gdf, districts[['ADM_NM','geometry']], how='left', predicate='within')
station_region_map = dict(zip(joined['정류소명'], joined['ADM_NM']))

# (D) 통합 그래프에 붙이기
for node, data in integrated_graph.nodes(data=True):
    if data.get('mode')=='bus':
        station = data['station']
        integrated_graph.nodes[node]['region'] = station_region_map.get(station, 'Unknown')


DataSourceError: seoul_districts.geojson: No such file or directory

In [92]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# 1) 정류장 정보 로드
stops_df = pd.read_excel('bus_stops.xlsx', dtype={'X좌표': float, 'Y좌표': float})
# 좌표 → Point 객체
stops_gdf = gpd.GeoDataFrame(
    stops_df,
    geometry=[Point(xy) for xy in zip(stops_df['X좌표'], stops_df['Y좌표'])],
    crs='EPSG:4326'  # WGS84
)

# 2) 서울시 구 경계 GeoJSON 로드 (미리 준비된 파일)
districts = gpd.read_file('seoul_districts.geojson')  # 'ADM_NM' 컬럼에 구 이름 있다고 가정
districts = districts.to_crs('EPSG:4326')

# 3) 공간 결합: 각 정류장이 속한 구를 찾아 병합
stops_with_region = gpd.sjoin(
    stops_gdf,
    districts[['ADM_NM', 'geometry']],
    how='left',
    predicate='within'
).rename(columns={'ADM_NM': 'region'})

# 4) station → region 매핑 딕셔너리 생성
station_region_map = dict(
    zip(
        stops_with_region['정류소명'],
        stops_with_region['region'].fillna('Unknown')
    )
)

# 5) integrated_graph 에 적용
for node, data in bus_graph.nodes(data=True):
    stop_name = data['station']  # 기존에 붙여둔 정류장명
    region    = station_region_map.get(stop_name, 'Unknown')
    integrated_graph.nodes[node]['region'] = region


DataSourceError: seoul_districts.geojson: No such file or directory

In [88]:
# bus_speed_season 에서 뽑아낸 DataFrame:
# columns = ['category','region','spring','summer','fall','winter']

# (region, season) → 속도 비율
region_speed_map = {
    (row['region'], season): row[season]
    for _, row in bus_speed_season.iterrows()
    for season in ['spring','summer','fall','winter']
}


# 여기서부터는 참고

In [ ]:
def get_least_transfer_routes_optimized(G, origin_candidates, dest_candidates, k=10, max_transfers=4):
    """
    - transfer_weight: 엣지별 “환승시 10000 패널티 + 통행시간” 으로 가중치 계산
    """
    def transfer_weight(u, v, edge_data):
        mode = edge_data.get('mode')
        w = edge_data.get('weight', 0)
        if mode == 'transfer':
            return 10000 + w
        if mode == 'metro_edge':
            lu = G.nodes[u].get('line')
            lv = G.nodes[v].get('line')
            if lu is None or lv is None or lu != lv:
                return 10000 + w
            return w
        if mode == 'bus_edge':
            return w
        return w

    least_transfer_list = []

    for origin_node, origin_walk in origin_candidates:
        # 1) origin_node 한 번에 Dijkstra → distances, paths 얻기
        distances, paths = nx.single_source_dijkstra(
            G,
            origin_node,
            weight=transfer_weight
        )

        for dest_node, dest_walk in dest_candidates:
            if dest_node not in distances:
                continue

            # (a) distances[dest_node]에는 “환승 패널티 포함 + 내부 통행시간”이 누적된 값
            # → 실제 여행 시간(내부 교통시간)만 따로 계산하려면 경로를 순회해야 함.
            path = paths[dest_node]

            # 2) 실제 환승 횟수 계산 (기존과 동일)
            transfer_count = 0
            prev_mode, prev_line = None, None
            edges = list(zip(path, path[1:]))

            for idx, (u, v) in enumerate(edges):
                ed = G[u][v]
                mode = ed.get('mode')
                if mode == 'transfer':
                    if idx != 0 and idx != (len(edges) - 1):
                        transfer_count += 1
                    prev_mode, prev_line = 'transfer', None
                    continue
                if mode == 'metro_edge':
                    cur_line = G.nodes[u].get('line')
                    if prev_mode == 'metro_edge' and prev_line != cur_line:
                        transfer_count += 1
                    prev_mode, prev_line = 'metro_edge', cur_line
                    continue
                if mode == 'bus_edge':
                    cur_br = next(iter(ed.get('lines', [])), None)
                    if prev_mode == 'bus_edge' and prev_line != cur_br:
                        transfer_count += 1
                    prev_mode, prev_line = 'bus_edge', cur_br
                    continue
                prev_mode, prev_line = mode, None

            if transfer_count > max_transfers:
                continue

            # 3) 실제 내부 통행시간(버스/지하철/환승 weight)은 경로를 순회하며 직접 합산
            travel_time = sum(G[u][v]['weight'] for u, v in edges)

            total_time = travel_time + origin_walk + dest_walk
            total_walk = origin_walk + dest_walk

            least_transfer_list.append({
                'path': path,
                'time_min': total_time,
                'transfers': transfer_count,
                'walk_min': total_walk
            })

    # 4) (환승 횟수, 총 시간) 기준으로 정렬
    least_transfer_list.sort(key=lambda x: (x['transfers'], x['time_min']))
    return least_transfer_list[:k]


In [ ]:
def get_least_walking_routes_optimized(G, origin_candidates, dest_candidates, k=10, max_transfers=4):
    def walk_only_weight(u, v, edge_data):
        return edge_data['weight'] if edge_data.get('mode') == 'transfer' else 0

    least_walking_list = []

    for origin_node, origin_walk in origin_candidates:
        # 1) origin_node 한 번에 Dijkstra → distances, paths
        distances, paths = nx.single_source_dijkstra(
            G,
            origin_node,
            weight=walk_only_weight
        )

        for dest_node, dest_walk in dest_candidates:
            if dest_node not in distances:
                continue

            # (a) distances[dest_node] = “경로 중 transfer weight 합”
            walk_length_along_graph = distances[dest_node]
            path = paths[dest_node]

            # 2) 환승 횟수 계산 (기존 로직과 동일)
            transfer_count = 0
            prev_mode, prev_line = None, None
            edges = list(zip(path, path[1:]))

            for idx, (u, v) in enumerate(edges):
                ed = G[u][v]
                mode = ed.get('mode')
                if mode == 'transfer':
                    if 0 < idx < (len(edges)-1):
                        transfer_count += 1
                    prev_mode, prev_line = 'transfer', None
                    continue
                if mode == 'metro_edge':
                    cur_line = G.nodes[u].get('line')
                    if prev_mode == 'metro_edge' and prev_line != cur_line:
                        transfer_count += 1
                    prev_mode, prev_line = 'metro_edge', cur_line
                    continue
                if mode == 'bus_edge':
                    cur_br = next(iter(ed.get('lines', [])), None)
                    if prev_mode == 'bus_edge' and prev_line != cur_br:
                        transfer_count += 1
                    prev_mode, prev_line = 'bus_edge', cur_br
                    continue
                prev_mode, prev_line = mode, None

            if transfer_count > max_transfers:
                continue

            total_walk = walk_length_along_graph + origin_walk + dest_walk
            travel_time = sum(G[u][v]['weight'] for u, v in edges)
            total_time = travel_time + origin_walk + dest_walk

            least_walking_list.append({
                'path': path,
                'time_min': total_time,
                'transfers': transfer_count,
                'walk_min': total_walk
            })

    least_walking_list.sort(key=lambda x: (x['walk_min'], x['time_min']))
    return least_walking_list[:k]


In [ ]:
def get_top_routes_optimized(G, origin_candidates, dest_candidates, k_per_criteria=10, max_transfers=4):
    return {
        'fastest': get_fastest_routes_optimized(G, origin_candidates, dest_candidates,
                                                k=k_per_criteria, max_transfers=max_transfers),
        'least_transfers': get_least_transfer_routes_optimized(G, origin_candidates, dest_candidates,
                                                                k=k_per_criteria, max_transfers=max_transfers),
        'least_walking': get_least_walking_routes_optimized(G, origin_candidates, dest_candidates,
                                                            k=k_per_criteria, max_transfers=max_transfers)
    }


In [ ]:
# 4) 실제 호출 예시:
results = get_top_routes_optimized(
    integrated_graph,
    start_nodes_with_walk,   # 반경 500m 이내 후보 + 도로망 기반 도보 시간
    end_nodes_with_walk,
    k_per_criteria=10,       # 각 기준별 상위 10개
    max_transfers=4
)

# 5) 결과 출력
for criterion, routes in results.items():
    print(f"=== {criterion.upper()} ROUTES (max_transfers=4) ===")
    for idx, route in enumerate(routes, 1):
        path_str = " → ".join(route['path'])
        print(f"[{idx}] 경로: {path_str}")
        print(f"     총시간: {route['time_min']:.1f}분 | 환승: {route['transfers']}회 | 도보: {route['walk_min']:.1f}분")
    print()


=== FASTEST ROUTES (max_transfers=4) ===
[1] 경로: 6호선_안암 → 6호선_보문 → 6호선_창신 → 6호선_동묘앞 → 6호선_신당 → 6호선_청구 → 6호선_약수 → 3호선_약수 → 3호선_금호 → 3호선_옥수 → 3호선_압구정 → 3호선_신사 → 신분당선_신사 → 신분당선_논현 → 신분당선_신논현 → 신분당선_강남
     총시간: 42.2분 | 환승: 2회 | 도보: 11.6분
[2] 경로: 6호선_고려대 → 6호선_안암 → 6호선_보문 → 6호선_창신 → 6호선_동묘앞 → 6호선_신당 → 6호선_청구 → 6호선_약수 → 3호선_약수 → 3호선_금호 → 3호선_옥수 → 3호선_압구정 → 3호선_신사 → 신분당선_신사 → 신분당선_논현 → 신분당선_신논현 → 신분당선_강남
     총시간: 44.6분 | 환승: 2회 | 도보: 12.1분
[3] 경로: 6호선_안암 → 6호선_보문 → 6호선_창신 → 6호선_동묘앞 → 6호선_신당 → 6호선_청구 → 6호선_약수 → 3호선_약수 → 3호선_금호 → 3호선_옥수 → 3호선_압구정 → 3호선_신사 → 신분당선_신사 → 신분당선_논현 → 신분당선_신논현 → 신분당선_강남 → 2호선_강남
     총시간: 47.8분 | 환승: 2회 | 도보: 12.6분
[4] 경로: bus_141_고려대역.고대앞삼거리 → bus_141_제기동한신아파트앞 → bus_141_홍파초등학교 → bus_141_경동시장앞 → bus_141_동대문구청.용신동주민센터 → bus_141_마장축산물시장 → bus_141_도선사거리 → bus_141_성동구청 → bus_141_무학여고앞 → bus_141_응봉사거리 → bus_141_뚝섬서울숲 → bus_141_성수대교남단.현대아파트 → bus_141_신구중학교 → bus_141_도산공원 → bus_141_서울세관 → bus_141_임피리얼팰리스호텔.펜트힐루논현 → bus_141_논현아이파크 → bus_141_시티프라디움더강남.라움아트센터 → bus_141_KT강남지사

########

In [ ]:
import requests
import xml.etree.ElementTree as ET  # API가 XML 응답일 경우
import pandas as pd
from IPython.display import display


서울교통공사 1-8호선 30분 단위 평균 혼잡도로 30분간 지나는 열차들의 평균 혼잡도(정원대비 승차인원으로, 승차인과 좌석수가 일치할 경우를 혼잡도 34%로 산정) 입니다.(단위: %).
서울교통공사 혼잡도 데이터는 요일구분(평일, 토요일, 일요일), 호선, 역번호, 역명, 상하선구분, 30분단위 별 혼잡도 데이터로 구성되어 있습니다. (2024년부터 분기별 제공됩니다.)

일반적인 시내버스(도시표준형)의 경우, 좌석수는 21석, 입석은 40석, 운전석을 포함하여 총 62명까지 탑승 가능합니다. 광역버스나 직행좌석버스는 좌석 수가 더 많거나 2층 버스의 경우 70석 이상이 될 수도 있습니다.


배차간격 10분.